# Practica 2. Pareja 02: Daniel Mateo y Laura Sánchez.

In [ ]:
# Cargamos todas las librerias necesarias para este Notebook.
from scipy.stats import norm
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import *
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *
from ClasificadorNB import *
from ClasificadorKNN import *
from ClasificadorRegresionLogistica import *
from Distancias import *
import math
from pylab import *
import matplotlib.pyplot as plt

## APARTADO 1
En este primer apartado hemos desarrollado el algoritmo de aprendizaje de vecinos próximos o KNN.
- Este algoritmo consiste en obtener los datos de entrenamiento y almacenarlos, normalizados o sin normalizar, para la clasificación. Por lo tanto, cuando se vaya a clasificar, este algoritmo va a ir dato a dato del set de Test y va a realizar un cálculo de la distancia entre ese dato y todos los demás datos almacenados en entrenamiento. Y, una vez tenemos los valores de las distancias a cada dato de Train, se procede a ordenar los datos de menor a mayor en función de la distancia, por lo que escoger los K vecinos más próximos será lo mismo a obtener los K datos de Train con menor distancia al dato actual del Test. Entonces, se obtiene la moda de la clase. Dependiendo de qué clase se repita más en cada grupo de vecinos, se le clasificará con esa clase al dato de Test. Y así se repite el procedimiento con el resto de datos de Test.


- Falta indicar que los datos de Test también tienen que estar normalizados si los de Train lo están. Para esto, hemos creado dentro de cada modelo (KNN y RegLog) las funciones calcularMediasDesv() que se encarga de calcular la media y la desviación del conjunto de datos pasado, y la función normalizarDatos() que se encarga de normalizar el set de datos con la media y la desviación calculadas en la otra función.


- Pero antes de nada, hay que leer ambos ficheros (pima-indians-diabetes.data y wdbc.data) para generar sus correspondientes clases de Datos. Estos datasets se van a leer en esta parte del fichero y se utilizarán a lo largo de este en otros apartados también. Si se desea comprobar que se han procesado bien los datasets, puede descomentar los prints. Estos son:

In [ ]:
# Dataset de pima-indians-data.data.
dataset_indians = Datos("./pima-indians-diabetes.data")
#print(dataset_indians.nominalAtributos)
#print(dataset_indians.diccionario)
#print(dataset_indians.datos)

# Dataset de wdbc.data.
dataset_wdbc = Datos("./wdbc.data")
#print(dataset_wdbc.nominalAtributos)
#print(dataset_wdbc.diccionario)
#print(dataset_wdbc.datos)

- Una vez tenemos los datasets generados, hemos creado la estrategia de particionado de Validación Cruzada de 4 particiones, que se va a utilizar en cada modelo de esta práctica (como se indicó en clase).

In [ ]:
# Validación Cruzada con K=4 particiones totales.
vc = ValidacionCruzada(4) 

- A continuación, creamos los objetos de las distintas métricas para las distancias que se pueden utilizar en el algoritmo de Vecinos Próximos. La primera es la Distancia Euclidea, cuya fórmula es sqrt( Sumi=1_N (x[i] - y[i])^2 ). La segunda es la Distancia Manhattan, cuya fórmula es Sumi=1_N | x[i] - y[i] |. Y, por último, la Distancia Mahalanobis, cuya fórmula es sqrt( (x - y)^T * Σ^-1 * (x - y) ) donde Σ^-1 es la matriz de covarianzas inversa de los datos de entrenamiento y T es vector transpuesto. En este problema, x sería el dato de Train e y el dato de Test.


- Estas distancias se encuentran como clases independientes dentro de Distancias.py y heredan de la clase Distancia, ya que poseen todas un método calculaDistancia() concreto, que se encarga de calcular, mediante los dos datos a comparar (más la matriz de covarianzas inversa en el caso de Mahalanobis), la distancia con la fórmula correspondiente antes nombrada.

In [ ]:
# Creamos las distancias a utilizar.
de = DistanciaEuclidea()
dmh = DistanciaManhattan()
dmb = DistanciaMahalanobis()

- Tras esto, ya se van a declarar y generar los distintos modelos de Vecinos Próximos que vamos a utilizar para validación y comparar sus resultados de tasas de error, en forma de medias y desviaciones de los resultados de las cuatro particiones de Validación Cruzada.


- Pero antes, vamos a declarar la estructura que almacenará estos valores de medias y desviaciones, la estructura con las matrices de confusión de los modelos, y la estructura con las tasas de confusión de los modelos, además de la función auxiliar que va a calcularlos de un array con tasas de error recibida. Estos se usarán en todos los apartados de este fichero.

In [ ]:
# Diccionario con las medias y desviaciones para las posteriores tablas.
dicc_med_desv = {}
dicc_med_desv['NB'] = {}

dicc_med_desv['KNN'] = {}
dicc_med_desv['KNN']['K1'] = {}
dicc_med_desv['KNN']['K5'] = {}
dicc_med_desv['KNN']['K11'] = {}
dicc_med_desv['KNN']['K21'] = {}
dicc_med_desv['KNN']['K1']['SN'] = {}
dicc_med_desv['KNN']['K1']['NN'] = {}
dicc_med_desv['KNN']['K5']['SN'] = {}
dicc_med_desv['KNN']['K5']['NN'] = {}
dicc_med_desv['KNN']['K11']['SN'] = {}
dicc_med_desv['KNN']['K11']['NN'] = {}
dicc_med_desv['KNN']['K21']['SN'] = {}
dicc_med_desv['KNN']['K21']['NN'] = {}

dicc_med_desv['REG_LOG'] = {}
dicc_med_desv['REG_LOG']['NE1'] = {}
dicc_med_desv['REG_LOG']['NE10'] = {}
dicc_med_desv['REG_LOG']['NE100'] = {}

# Diccionario con las matrices de confusión para las posteriores tablas.
dicc_matrices = {}
dicc_matrices['NB'] = {}

dicc_matrices['KNN'] = {}
dicc_matrices['KNN']['K1'] = {}
dicc_matrices['KNN']['K5'] = {}
dicc_matrices['KNN']['K11'] = {}
dicc_matrices['KNN']['K21'] = {}
dicc_matrices['KNN']['K1']['SN'] = {}
dicc_matrices['KNN']['K1']['NN'] = {}
dicc_matrices['KNN']['K5']['SN'] = {}
dicc_matrices['KNN']['K5']['NN'] = {}
dicc_matrices['KNN']['K11']['SN'] = {}
dicc_matrices['KNN']['K11']['NN'] = {}
dicc_matrices['KNN']['K21']['SN'] = {}
dicc_matrices['KNN']['K21']['NN'] = {}

dicc_matrices['REG_LOG'] = {}
dicc_matrices['REG_LOG']['NE1'] = {}
dicc_matrices['REG_LOG']['NE10'] = {}
dicc_matrices['REG_LOG']['NE100'] = {}

# Diccionario con las tasas de confusión para el posterior análisis ROC (solo los mejores obtenidos se compararán).
dicc_tasas = {}
dicc_tasas['NB'] = {}
dicc_tasas['KNN'] = {}
dicc_tasas['REG_LOG'] = {}

# Función auxiliar que calcula la media y la desviacion de las tasas de error de un modelo.
def calcula_med_desv(tasas_error, rev):
    n = len(tasas_error)
    media = 0
    desviacion = 0
    for error in tasas_error:
        if rev:
            error = 1 - error
        media += error
    media /= n
    
    for error in tasas_error:
        if rev:
            error = 1 - error
        desviacion += (error - media)**2
    desviacion = math.sqrt(desviacion/n)
    
    return media, desviacion;

- Ahora sí, generamos los distintos modelos que vamos a probar y comparar. Principalmente, crearemos uno por cada valor distinto de K como 1, 5, 11 y 21, por cada forma distinta de tratar los datos, es decir, normalizados o sin normalizar. Y, además, se probará con cada tipo de métrica de distancia. Por lo tanto:

In [ ]:
# Clasificador de KNN Distancia Euclidea con K = 1 y datos normalizados
knn_de_k1_norm = ClasificadorKNN(1, de, True)
# Clasificador de KNN Distancia Euclidea con K = 5 y datos normalizados
knn_de_k5_norm = ClasificadorKNN(5, de, True)
# Clasificador de KNN Distancia Euclidea con K = 11 y datos normalizados
knn_de_k11_norm = ClasificadorKNN(11, de, True)
# Clasificador de KNN Distancia Euclidea con K = 21 y datos normalizados
knn_de_k21_norm = ClasificadorKNN(21, de, True)

# Clasificador de KNN Distancia Manhattan con K = 1 y datos normalizados
knn_dmh_k1_norm = ClasificadorKNN(1, dmh, True)
# Clasificador de KNN Distancia Euclidea con K = 5 y datos normalizados
knn_dmh_k5_norm = ClasificadorKNN(5, dmh, True)
# Clasificador de KNN Distancia Euclidea con K = 11 y datos normalizados
knn_dmh_k11_norm = ClasificadorKNN(11, dmh, True)
# Clasificador de KNN Distancia Euclidea con K = 21 y datos normalizados
knn_dmh_k21_norm = ClasificadorKNN(21, dmh, True)

# Clasificador de KNN Distancia Mahalanobis con K = 1 y datos normalizados
knn_dmb_k1_norm = ClasificadorKNN(1, dmb, True)
# Clasificador de KNN Distancia Mahalanobis  con K = 5 y datos normalizados
knn_dmb_k5_norm = ClasificadorKNN(5, dmb, True)
# Clasificador de KNN Distancia Mahalanobis  con K = 11 y datos normalizados
knn_dmb_k11_norm = ClasificadorKNN(11, dmb, True)
# Clasificador de KNN Distancia Mahalanobis  con K = 21 y datos normalizados
knn_dmb_k21_norm = ClasificadorKNN(21, dmb, True)

# Clasificador de KNN Distancia Euclidea con K = 1 y datos sin normalizar
knn_de_k1 = ClasificadorKNN(1, de, False)
# Clasificador de KNN Distancia Euclidea con K = 5 y datos sin normalizar
knn_de_k5 = ClasificadorKNN(5, de, False)
# Clasificador de KNN Distancia Euclidea con K = 11 y datos sin normalizar
knn_de_k11 = ClasificadorKNN(11, de, False)
# Clasificador de KNN Distancia Euclidea con K = 21 y datos sin normalizar
knn_de_k21 = ClasificadorKNN(21, de, False)

# Clasificador de KNN Distancia Manhattan con K = 1 y datos sin normalizar
knn_dmh_k1 = ClasificadorKNN(1, dmh, False)
# Clasificador de KNN Distancia Euclidea con K = 5 y datos sin normalizar
knn_dmh_k5 = ClasificadorKNN(5, dmh, False)
# Clasificador de KNN Distancia Euclidea con K = 11 y datos sin normalizar
knn_dmh_k11 = ClasificadorKNN(11, dmh, False)
# Clasificador de KNN Distancia Euclidea con K = 21 y datos sin normalizar
knn_dmh_k21 = ClasificadorKNN(21, dmh, False)

# Clasificador de KNN Distancia Mahalanobis con K = 1 y datos sin normalizar
knn_dmb_k1 = ClasificadorKNN(1, dmb, False)
# Clasificador de KNN Distancia Mahalanobis  con K = 5 y datos sin normalizar
knn_dmb_k5 = ClasificadorKNN(5, dmb, False)
# Clasificador de KNN Distancia Mahalanobis  con K = 11 y datos sin normalizar
knn_dmb_k11 = ClasificadorKNN(11, dmb, False)
# Clasificador de KNN Distancia Mahalanobis  con K = 21 y datos sin normalizar
knn_dmb_k21 = ClasificadorKNN(21, dmb, False)

- Entonces, ya podemos realizar el proceso de entrenamiento y de clasificación de cada uno de los modelos creados antes y apuntar cada media y desviación en su correspondiente lugar del diccionario, además de su matriz de confusión y tasas de confusión. Hay muchos datos obtenidos, por lo que más adelante se volcará todo a una tabla para que pueda verlo mejor. Por lo tanto:

In [ ]:
# Validaciones de los distintos modelos de Ks para cada dataset con Distancia Euclidea
dicc_med_desv['KNN']['K1']['SN']['DE'] = {}
dicc_med_desv['KNN']['K5']['SN']['DE'] = {}
dicc_med_desv['KNN']['K11']['SN']['DE'] = {}
dicc_med_desv['KNN']['K21']['SN']['DE'] = {}
dicc_med_desv['KNN']['K1']['NN']['DE'] = {}
dicc_med_desv['KNN']['K5']['NN']['DE'] = {}
dicc_med_desv['KNN']['K11']['NN']['DE'] = {}
dicc_med_desv['KNN']['K21']['NN']['DE'] = {}
dicc_matrices['KNN']['K1']['SN']['DE'] = {}
dicc_matrices['KNN']['K1']['NN']['DE'] = {}
dicc_matrices['KNN']['K5']['SN']['DE'] = {}
dicc_matrices['KNN']['K5']['NN']['DE'] = {}
dicc_matrices['KNN']['K11']['SN']['DE'] = {}
dicc_matrices['KNN']['K11']['NN']['DE'] = {}
dicc_matrices['KNN']['K21']['SN']['DE'] = {}
dicc_matrices['KNN']['K21']['NN']['DE'] = {}

# Dataset Indians K = 1 Datos Normalizados
tasas_error = knn_de_k1_norm.validacion(vc, dataset_indians, knn_de_k1_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K1']['SN']['DE']['I'] = knn_de_k1_norm.matriz_confusion

# Dataset WDBC K = 1 Datos Normalizados
tasas_error = knn_de_k1_norm.validacion(vc, dataset_wdbc, knn_de_k1_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K1']['SN']['DE']['W'] = knn_de_k1_norm.matriz_confusion

# Dataset Indians K = 5 Datos Normalizados
tasas_error = knn_de_k5_norm.validacion(vc, dataset_indians, knn_de_k5_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K5']['SN']['DE']['I'] = knn_de_k5_norm.matriz_confusion

# Dataset WDBC K = 5 Datos Normalizados
tasas_error = knn_de_k5_norm.validacion(vc, dataset_wdbc, knn_de_k5_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K5']['SN']['DE']['W'] = knn_de_k5_norm.matriz_confusion

# Dataset Indians K = 11 Datos Normalizados
tasas_error = knn_de_k11_norm.validacion(vc, dataset_indians, knn_de_k11_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K11']['SN']['DE']['I'] = knn_de_k11_norm.matriz_confusion

# Dataset WDBC K = 11 Datos Normalizados
tasas_error = knn_de_k11_norm.validacion(vc, dataset_wdbc, knn_de_k11_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K11']['SN']['DE']['W'] = knn_de_k11_norm.matriz_confusion

# Dataset Indians K = 21 Datos Normalizados
tasas_error = knn_de_k21_norm.validacion(vc, dataset_indians, knn_de_k21_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K21']['SN']['DE']['I'] = knn_de_k21_norm.matriz_confusion
dicc_tasas['KNN']['I'] = knn_de_k21_norm.tasas_confusion

# Dataset WDBC K = 21 Datos Normalizados
tasas_error = knn_de_k21_norm.validacion(vc, dataset_wdbc, knn_de_k21_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K21']['SN']['DE']['W'] = knn_de_k21_norm.matriz_confusion
dicc_tasas['KNN']['W'] = knn_de_k21_norm.tasas_confusion

# Dataset Indians K = 1 Datos Sin Normalizar
tasas_error = knn_de_k1.validacion(vc, dataset_indians, knn_de_k1, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K1']['NN']['DE']['I'] = knn_de_k1.matriz_confusion

# Dataset WDBC K = 1 Datos Sin Normalizar
tasas_error = knn_de_k1.validacion(vc, dataset_wdbc, knn_de_k1, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K1']['NN']['DE']['W'] = knn_de_k1.matriz_confusion

# Dataset Indians K = 5 Datos Sin Normalizar
tasas_error = knn_de_k5.validacion(vc, dataset_indians, knn_de_k5, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K5']['NN']['DE']['I'] = knn_de_k5.matriz_confusion

# Dataset WDBC K = 5 Datos Sin Normalizar
tasas_error = knn_de_k5.validacion(vc, dataset_wdbc, knn_de_k5, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K5']['NN']['DE']['W'] = knn_de_k5.matriz_confusion

# Dataset Indians K = 11 Datos Sin Normalizar
tasas_error = knn_de_k11.validacion(vc, dataset_indians, knn_de_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K11']['NN']['DE']['I'] = knn_de_k11.matriz_confusion

# Dataset WDBC K = 11 Datos Sin Normalizar
tasas_error = knn_de_k11.validacion(vc, dataset_wdbc, knn_de_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K11']['NN']['DE']['W'] = knn_de_k11.matriz_confusion

# Dataset Indians K = 21 Datos Sin Normalizar
tasas_error = knn_de_k21.validacion(vc, dataset_indians, knn_de_k21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_I'] = desviacion
dicc_matrices['KNN']['K21']['NN']['DE']['I'] = knn_de_k21.matriz_confusion

# Dataset WDBC K = 21 Datos Sin Normalizar
tasas_error = knn_de_k21.validacion(vc, dataset_wdbc, knn_de_k21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_W'] = desviacion
dicc_matrices['KNN']['K21']['NN']['DE']['W'] = knn_de_k21.matriz_confusion

In [ ]:
# Validaciones de los distintos modelos de Ks para cada dataset con Distancia Manhattan
dicc_med_desv['KNN']['K1']['SN']['DMH'] = {}
dicc_med_desv['KNN']['K5']['SN']['DMH'] = {}
dicc_med_desv['KNN']['K11']['SN']['DMH'] = {}
dicc_med_desv['KNN']['K21']['SN']['DMH'] = {}
dicc_med_desv['KNN']['K1']['NN']['DMH'] = {}
dicc_med_desv['KNN']['K5']['NN']['DMH'] = {}
dicc_med_desv['KNN']['K11']['NN']['DMH'] = {}
dicc_med_desv['KNN']['K21']['NN']['DMH'] = {}
dicc_matrices['KNN']['K1']['SN']['DMH'] = {}
dicc_matrices['KNN']['K1']['NN']['DMH'] = {}
dicc_matrices['KNN']['K5']['SN']['DMH'] = {}
dicc_matrices['KNN']['K5']['NN']['DMH'] = {}
dicc_matrices['KNN']['K11']['SN']['DMH'] = {}
dicc_matrices['KNN']['K11']['NN']['DMH'] = {}
dicc_matrices['KNN']['K21']['SN']['DMH'] = {}
dicc_matrices['KNN']['K21']['NN']['DMH'] = {}

# Dataset Indians K = 1 Datos Normalizados
tasas_error = knn_dmh_k1_norm.validacion(vc, dataset_indians, knn_dmh_k1_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K1']['SN']['DMH']['I'] = knn_dmh_k1_norm.matriz_confusion

# Dataset WDBC K = 1 Datos Normalizados
tasas_error = knn_dmh_k1_norm.validacion(vc, dataset_wdbc, knn_dmh_k1_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K1']['SN']['DMH']['W'] = knn_dmh_k1_norm.matriz_confusion

# Dataset Indians K = 5 Datos Normalizados
tasas_error = knn_dmh_k5_norm.validacion(vc, dataset_indians, knn_dmh_k5_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K5']['SN']['DMH']['I'] = knn_dmh_k5_norm.matriz_confusion

# Dataset WDBC K = 5 Datos Normalizados
tasas_error = knn_dmh_k5_norm.validacion(vc, dataset_wdbc, knn_dmh_k5_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K5']['SN']['DMH']['W'] = knn_dmh_k5_norm.matriz_confusion

# Dataset Indians K = 11 Datos Normalizados
tasas_error = knn_dmh_k11_norm.validacion(vc, dataset_indians, knn_dmh_k11_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K11']['SN']['DMH']['I'] = knn_dmh_k11_norm.matriz_confusion

# Dataset WDBC K = 11 Datos Normalizados
tasas_error = knn_dmh_k11_norm.validacion(vc, dataset_wdbc, knn_dmh_k11_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K11']['SN']['DMH']['W'] = knn_dmh_k11_norm.matriz_confusion

# Dataset Indians K = 21 Datos Normalizados
tasas_error = knn_dmh_k21_norm.validacion(vc, dataset_indians, knn_dmh_k21_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K21']['SN']['DMH']['I'] = knn_dmh_k21_norm.matriz_confusion

# Dataset WDBC K = 21 Datos Normalizados
tasas_error = knn_dmh_k21_norm.validacion(vc, dataset_wdbc, knn_dmh_k21_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K21']['SN']['DMH']['W'] = knn_dmh_k21_norm.matriz_confusion

# Dataset Indians K = 1 Datos Sin Normalizar
tasas_error = knn_dmh_k1.validacion(vc, dataset_indians, knn_dmh_k1, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K1']['NN']['DMH']['I'] = knn_dmh_k1.matriz_confusion

# Dataset WDBC K = 1 Datos Sin Normalizar
tasas_error = knn_dmh_k1.validacion(vc, dataset_wdbc, knn_dmh_k1, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K1']['NN']['DMH']['W'] = knn_dmh_k1.matriz_confusion

# Dataset Indians K = 5 Datos Sin Normalizar
tasas_error = knn_dmh_k5.validacion(vc, dataset_indians, knn_dmh_k5, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K5']['NN']['DMH']['I'] = knn_dmh_k5.matriz_confusion

# Dataset WDBC K = 5 Datos Sin Normalizar
tasas_error = knn_dmh_k5.validacion(vc, dataset_wdbc, knn_dmh_k5, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K5']['NN']['DMH']['W'] = knn_dmh_k5.matriz_confusion

# Dataset Indians K = 11 Datos Sin Normalizar
tasas_error = knn_dmh_k11.validacion(vc, dataset_indians, knn_dmh_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K11']['NN']['DMH']['I'] = knn_dmh_k11.matriz_confusion

# Dataset WDBC K = 11 Datos Sin Normalizar
tasas_error = knn_dmh_k11.validacion(vc, dataset_wdbc, knn_dmh_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K11']['NN']['DMH']['W'] = knn_dmh_k11.matriz_confusion

# Dataset Indians K = 21 Datos Sin Normalizar
tasas_error = knn_dmh_k21.validacion(vc, dataset_indians, knn_dmh_k21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_I'] = desviacion
dicc_matrices['KNN']['K21']['NN']['DMH']['I'] = knn_dmh_k21.matriz_confusion

# Dataset WDBC K = 21 Datos Sin Normalizar
tasas_error = knn_dmh_k11.validacion(vc, dataset_wdbc, knn_dmh_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_W'] = desviacion
dicc_matrices['KNN']['K21']['NN']['DMH']['W'] = knn_dmh_k11.matriz_confusion

In [ ]:
# Validaciones de los distintos modelos de Ks para cada dataset con Distancia Mahalanobis
dicc_med_desv['KNN']['K1']['SN']['DMB'] = {}
dicc_med_desv['KNN']['K5']['SN']['DMB'] = {}
dicc_med_desv['KNN']['K11']['SN']['DMB'] = {}
dicc_med_desv['KNN']['K21']['SN']['DMB'] = {}
dicc_med_desv['KNN']['K1']['NN']['DMB'] = {}
dicc_med_desv['KNN']['K5']['NN']['DMB'] = {}
dicc_med_desv['KNN']['K11']['NN']['DMB'] = {}
dicc_med_desv['KNN']['K21']['NN']['DMB'] = {}
dicc_matrices['KNN']['K1']['SN']['DMB'] = {}
dicc_matrices['KNN']['K1']['NN']['DMB'] = {}
dicc_matrices['KNN']['K5']['SN']['DMB'] = {}
dicc_matrices['KNN']['K5']['NN']['DMB'] = {}
dicc_matrices['KNN']['K11']['SN']['DMB'] = {}
dicc_matrices['KNN']['K11']['NN']['DMB'] = {}
dicc_matrices['KNN']['K21']['SN']['DMB'] = {}
dicc_matrices['KNN']['K21']['NN']['DMB'] = {}

# Dataset Indians K = 1 Datos Normalizados
tasas_error = knn_dmb_k1_norm.validacion(vc, dataset_indians, knn_dmb_k1_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K1']['SN']['DMB']['I'] = knn_dmb_k1_norm.matriz_confusion

# Dataset WDBC K = 1 Datos Normalizados
tasas_error = knn_dmb_k1_norm.validacion(vc, dataset_wdbc, knn_dmb_k1_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K1']['SN']['DMB']['W'] = knn_dmb_k1_norm.matriz_confusion

# Dataset Indians K = 5 Datos Normalizados
tasas_error = knn_dmb_k5_norm.validacion(vc, dataset_indians, knn_dmb_k5_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K5']['SN']['DMB']['I'] = knn_dmb_k5_norm.matriz_confusion

# Dataset WDBC K = 5 Datos Normalizados
tasas_error = knn_dmb_k5_norm.validacion(vc, dataset_wdbc, knn_dmb_k5_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K5']['SN']['DMB']['W'] = knn_dmb_k5_norm.matriz_confusion

# Dataset Indians K = 11 Datos Normalizados
tasas_error = knn_dmb_k11_norm.validacion(vc, dataset_indians, knn_dmb_k11_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K11']['SN']['DMB']['I'] = knn_dmb_k11_norm.matriz_confusion

# Dataset WDBC K = 11 Datos Normalizados
tasas_error = knn_dmb_k11_norm.validacion(vc, dataset_wdbc, knn_dmb_k11_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K11']['SN']['DMB']['W'] = knn_dmb_k11_norm.matriz_confusion

# Dataset Indians K = 21 Datos Normalizados
tasas_error = knn_dmb_k21_norm.validacion(vc, dataset_indians, knn_dmb_k21_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K21']['SN']['DMB']['I'] = knn_dmb_k21_norm.matriz_confusion

# Dataset WDBC K = 21 Datos Normalizados
tasas_error = knn_dmb_k21_norm.validacion(vc, dataset_wdbc, knn_dmb_k21_norm, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K21']['SN']['DMB']['W'] = knn_dmb_k21_norm.matriz_confusion

# Dataset Indians K = 1 Datos Sin Normalizar
tasas_error = knn_dmb_k1.validacion(vc, dataset_indians, knn_dmb_k1, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K1']['NN']['DMB']['I'] = knn_dmb_k1.matriz_confusion

# Dataset WDBC K = 1 Datos Sin Normalizar
tasas_error = knn_dmb_k1.validacion(vc, dataset_wdbc, knn_dmb_k1, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K1']['NN']['DMB']['W'] = knn_dmb_k1.matriz_confusion

# Dataset Indians K = 5 Datos Sin Normalizar
tasas_error = knn_dmb_k5.validacion(vc, dataset_indians, knn_dmb_k5, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K5']['NN']['DMB']['I'] = knn_dmb_k5.matriz_confusion

# Dataset WDBC K = 5 Datos Sin Normalizar
tasas_error = knn_dmb_k5.validacion(vc, dataset_wdbc, knn_dmb_k5, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K5']['NN']['DMB']['W'] = knn_dmb_k5.matriz_confusion

# Dataset Indians K = 11 Datos Sin Normalizar
tasas_error = knn_dmb_k11.validacion(vc, dataset_indians, knn_dmb_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K11']['NN']['DMB']['I'] = knn_dmb_k11.matriz_confusion

# Dataset WDBC K = 11 Datos Sin Normalizar
tasas_error = knn_dmb_k11.validacion(vc, dataset_wdbc, knn_dmb_k11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K11']['NN']['DMB']['W'] = knn_dmb_k11.matriz_confusion

# Dataset Indians K = 21 Datos Sin Normalizar
tasas_error = knn_dmb_k21.validacion(vc, dataset_indians, knn_dmb_k21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_I'] = desviacion
dicc_matrices['KNN']['K21']['NN']['DMB']['I'] = knn_dmb_k21.matriz_confusion

# Dataset WDBC K = 21 Datos Sin Normalizar
tasas_error = knn_dmb_k21.validacion(vc, dataset_wdbc, knn_dmb_k21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_W'] = desviacion
dicc_matrices['KNN']['K21']['NN']['DMB']['W'] = knn_dmb_k21.matriz_confusion

- Tras ejecutar todo lo anterior, se puede ver las siguientes tablas que almacenan las medias y desviaciones de las validaciones de cada modelo anterior para cada dataset existente. (Es posible que lo anterior le lleve un tiempo ejecutarse, ya que son muchas validaciones de manera secuencial).

In [ ]:
#print(dicc_med_desv)
# *Forma tabla*
# Dataset_x_norm:        | K1_μ | K1_σ | K5_μ | K5_σ |...
#                      DE|      |      |      |      |...
#                     DMH|      |      |      |      |...
#                     DMB|      |      |      |      |...
# DE == Se usa estrategia Distancia Euclidea
# DMH == Se usa estrategia Distancia Manhattan
# DMB == Se usa estrategia Distancia Mahalanobis
# KN_μ == Media μ de las tasas de error sobre modelo con K = N
# KN_σ == Desviación típica σ de las tasas de error sobre modelo con K = N
# x == Dataset Indians o WDBC
# norm == Datos normalizados

print("> Tabla con medias y desviaciones sobre dataset Indians con datos normalizados\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DE']['MED_I'] , dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_I'], dicc_med_desv['KNN']['K5']['SN']['DE']['MED_I'], dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_I'], dicc_med_desv['KNN']['K11']['SN']['DE']['MED_I'], dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_I'], dicc_med_desv['KNN']['K21']['SN']['DE']['MED_I'], dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_I'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_I'] , dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_I'], dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_I'], dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_I'], dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_I'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_I'] , dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_I'], dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_I'], dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_I'], dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_I'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset Indians sin datos normalizados\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DE']['MED_I'] , dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_I'], dicc_med_desv['KNN']['K5']['NN']['DE']['MED_I'], dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_I'], dicc_med_desv['KNN']['K11']['NN']['DE']['MED_I'], dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_I'], dicc_med_desv['KNN']['K21']['NN']['DE']['MED_I'], dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_I'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_I'] , dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_I'], dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_I'], dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_I'], dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_I'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_I'] , dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_I'], dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_I'], dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_I'], dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_I'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset WDBC con datos normalizados\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DE']['MED_W'] , dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_W'], dicc_med_desv['KNN']['K5']['SN']['DE']['MED_W'], dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_W'], dicc_med_desv['KNN']['K11']['SN']['DE']['MED_W'], dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_W'], dicc_med_desv['KNN']['K21']['SN']['DE']['MED_W'], dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_W'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_W'] , dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_W'], dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_W'], dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_W'], dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_W'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_W'] , dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_W'], dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_W'], dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_W'], dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_W'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset WDBC sin datos normalizados\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DE']['MED_W'] , dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_W'], dicc_med_desv['KNN']['K5']['NN']['DE']['MED_W'], dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_W'], dicc_med_desv['KNN']['K11']['NN']['DE']['MED_W'], dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_W'], dicc_med_desv['KNN']['K21']['NN']['DE']['MED_W'], dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_W'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_W'] , dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_W'], dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_W'], dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_W'], dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_W'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_W'] , dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_W'], dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_W'], dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_W'], dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_W'], '|'))


- En general, hemos observado que aumentar el valor de vecinos K mejora las clasificaciones del algoritmo, ya que se obtiene una media de tasa de error menor a medida que K aumenta. Esto es debido a que, si, por ejemplo, se mira 1 único vecino el clasificador va a fallar muy fácilmente, ya que solo dependes del valor de su clase (se podría ver como un 50-50 de probabilidades de que fuera una de las dos clases) y la clasificación es menos precisa. En cambio, si se tienen en cuenta más vecinos, como 21 por ejemplo, al decidir la clase del dato se tiene en cuenta la moda de la clase de todos ellos, por lo que la clase obtenida será más precisa al observar más datos de su entorno (comparten misma área en el espacio de datos).


- Además, generalmente, las métricas de distancias Euclidea y Manhattan dan una tasa de error menor respecto de la tasa de error de la distancia Mahalanobis. Sin embargo, creemos que esto no debería ser así porque las dos primeras tienen en cuenta únicamente la distancia entre ambos datos en el espacio, pero la Mahalanobis obtiene información de la distribución de los datos en el espacio mediante la matriz de covarianzas, por lo que, además de la distancia entre ambos puntos, tiene en cuenta cómo varían los puntos respecto a la distribución. Por lo tanto, cuando se utiliza la distancia Mahalanobis se debería realizar una clasificación más precisa que con las otras dos distancias, pero obtenemos un error medio mayor en esta métrica.


- En cuanto a la diferencia entre normalizar los datos o no, hemos observado que sale mayor tasa media de error cuando no se aplica este preprocesado en los datos de los datasets. Esto creemos que es debido porque hay atributos que no pesan lo mismo que otros al poseer valores mucho más grandes o pequeños que los demás. Esto afecta en la distancia, ya que estos atributos mayores afectan más y hacen que se ignoren los atributos pequeños, creando cierto sesgo, tendiendo más hacia el valor de ese atributo grande. Por lo tanto, al clasificar, esas distancias solo habrán tenido en cuenta los atributos más grandes, por lo que al obtener los vecinos, los más cercanos lo serán solo en esos atributos, pero puede que sean muy lejanos en los otros atributos. Si se normalizan los datos, se tienen en cuenta todos los atributos por igual al ser proporcionales en peso, por lo que la clasificación es más precisa.


- Hemos observado que obtenemos mejores resultados con el dataset wdbc y creemos que esto se debe a que los puntos de una misma clase se encuentran más cercanos unos de otros, es decir, la distribución de los datos de clase 1 está en una zona diferenciable a la distribución de los datos de la clase 2. Por lo tanto, cuando se realiza la clasificación, debido a cómo son las distribuciones, un dato de test obtendrá menores distancias siempre con los datos de su misma clase, al pertenecer a la misma distribución. Además, pensamos que KNN podría ser el modelo ideal para este problema.

## APARTADO 2
En este segundo apartado hemos desarrollado el modelo de aprendizaje lineal de Regresión Logística mediante Máxima Verosimilitud.

- Este algoritmo consiste en obtener los datos de entrenamiento y utilizarlos para actualizar un array de pesos llamada w, que es la que representa la frontera de decisión en el espacio de datos. Ese vector w se inicializará en entrenamiento con valores dentro del rango [-0.5, 0.5], como hemos visto en teoría. Luego, se iterará por número de épocas, donde en cada época se itera por cada dato del set de datos de Train y se aplica una fórmula para actualizar los valores del vector w, que sería w --> w - η*(sigma - t)*x, siendo t la clase del dato x, η la constante de aprendizaje elegida y sigma la probabilidad a posteriori de clase 1 del dato x. Esta última se obtiene el valor del producto escalar entre el actual w y el dato de entrenamiento x, y después la aplicación de la función sigmoidal (1/(1 + e^(-prod))) sobre ese valor, dando como resultado el valor sigma. Luego, en clasificación, simplemente se recorre el dataset de Test y se realiza un producto vectorial entre el vector w entrenado y el dato de test, resultando un valor que si es mayor que 0 se clasifica al dato como Clase 1.


- Los datos es conveniente que estén normalizados, por lo que en este apartado siempre vamos a normalizar los datos de entrenamiento y de test, aunque hemos hecho posible la opción de no normalizar en el modelo.


- A continuación, inicializamos y creamos los distintos modelos de Regresión Logística que posteriormente vamos a validar con ambos datasets y comparar sus resultados de tasas de error (como hemos hecho en el anterior apartado).

In [ ]:
# Clasificador de Regresion Logistica con η = 1 y Népocas = 1
rg_11 = ClasificadorRegresionLogistica(1, 1, True)
# Clasificador de Regresion Logistica con η = 1 y Népocas = 10
rg_110 = ClasificadorRegresionLogistica(10, 1, True)
# Clasificador de Regresion Logistica con η = 1 y Népocas = 100
rg_1100 = ClasificadorRegresionLogistica(100, 1, True)

# Clasificador de Regresion Logistica con η = 2 y Népocas = 1
rg_21 = ClasificadorRegresionLogistica(1, 2, True)
# Clasificador de Regresion Logistica con η = 2 y Népocas = 10
rg_210 = ClasificadorRegresionLogistica(10, 2, True)
# Clasificador de Regresion Logistica con η = 2 y Népocas = 100
rg_2100 = ClasificadorRegresionLogistica(100, 2, True)

# Clasificador de Regresion Logistica con η = 0.5 y Népocas = 1
rg_51 = ClasificadorRegresionLogistica(1, 0.5, True)
# Clasificador de Regresion Logistica con η = 0.5 y Népocas = 10
rg_510 = ClasificadorRegresionLogistica(10, 0.5, True)
# Clasificador de Regresion Logistica con η = 0.5 y Népocas = 100
rg_5100 = ClasificadorRegresionLogistica(100, 0.5, True)


- Vamos a probar con modelos de Regresión Logística con distintos valores de constante de aprendizaje η como 0,5, 1 y 2; y con distintos números de épocas como 1, 5, 10 y 100; por cada uno de los datasets existentes. Todos estos valores los almacenamos en la misma estructura que el apartado 1. Por lo tanto:

In [ ]:
# Validaciones de los distintos modelos de Regresión para cada dataset
dicc_med_desv['REG_LOG']['NE1']['N1'] = {}
dicc_med_desv['REG_LOG']['NE1']['N2'] = {}
dicc_med_desv['REG_LOG']['NE1']['N5'] = {}
dicc_med_desv['REG_LOG']['NE10']['N1'] = {}
dicc_med_desv['REG_LOG']['NE10']['N2'] = {}
dicc_med_desv['REG_LOG']['NE10']['N5'] = {}
dicc_med_desv['REG_LOG']['NE100']['N1'] = {}
dicc_med_desv['REG_LOG']['NE100']['N2'] = {}
dicc_med_desv['REG_LOG']['NE100']['N5'] = {}
dicc_matrices['REG_LOG']['NE1']['N1'] = {}
dicc_matrices['REG_LOG']['NE1']['N2'] = {}
dicc_matrices['REG_LOG']['NE1']['N5'] = {}
dicc_matrices['REG_LOG']['NE10']['N1'] = {}
dicc_matrices['REG_LOG']['NE10']['N2'] = {}
dicc_matrices['REG_LOG']['NE10']['N5'] = {}
dicc_matrices['REG_LOG']['NE100']['N1'] = {}
dicc_matrices['REG_LOG']['NE100']['N2'] = {}
dicc_matrices['REG_LOG']['NE100']['N5'] = {}

# Dataset Indians η = 1 y Népocas = 1
tasas_error = rg_11.validacion(vc, dataset_indians, rg_11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE1']['N1']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE1']['N1']['I'] = rg_11.matriz_confusion

# Dataset WDBC η = 1 y Népocas = 1
tasas_error = rg_11.validacion(vc, dataset_wdbc, rg_11, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE1']['N1']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE1']['N1']['W'] = rg_11.matriz_confusion

# Dataset Indians η = 2 y Népocas = 1
tasas_error = rg_21.validacion(vc, dataset_indians, rg_21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE1']['N2']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE1']['N2']['I'] = rg_21.matriz_confusion

# Dataset WDBC η = 2 y Népocas = 1
tasas_error = rg_21.validacion(vc, dataset_wdbc, rg_21, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE1']['N2']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE1']['N2']['W'] = rg_21.matriz_confusion

# Dataset Indians η = 0.5 y Népocas = 1
tasas_error = rg_51.validacion(vc, dataset_indians, rg_51, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE1']['N5']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE1']['N5']['I'] = rg_51.matriz_confusion

# Dataset WDBC η = 0.5 y Népocas = 1
tasas_error = rg_51.validacion(vc, dataset_wdbc, rg_51, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE1']['N5']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE1']['N5']['W'] = rg_51.matriz_confusion


# Dataset Indians η = 1 y Népocas = 10
tasas_error = rg_110.validacion(vc, dataset_indians, rg_110, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE10']['N1']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE10']['N1']['I'] = rg_110.matriz_confusion

# Dataset WDBC η = 1 y Népocas = 10
tasas_error = rg_110.validacion(vc, dataset_wdbc, rg_110, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE10']['N1']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE10']['N1']['W'] = rg_110.matriz_confusion

# Dataset Indians η = 2 y Népocas = 10
tasas_error = rg_210.validacion(vc, dataset_indians, rg_210, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE10']['N2']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE10']['N2']['I'] = rg_210.matriz_confusion

# Dataset WDBC η = 2 y Népocas = 10
tasas_error = rg_210.validacion(vc, dataset_wdbc, rg_210, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE10']['N2']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE10']['N2']['W'] = rg_210.matriz_confusion

# Dataset Indians η = 0.5 y Népocas = 10
tasas_error = rg_510.validacion(vc, dataset_indians, rg_510, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE10']['N5']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE10']['N5']['I'] = rg_510.matriz_confusion
dicc_tasas['REG_LOG']['I'] = rg_510.tasas_confusion

# Dataset WDBC η = 0.5 y Népocas = 10
tasas_error = rg_510.validacion(vc, dataset_wdbc, rg_510, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE10']['N5']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE10']['N5']['W'] = rg_510.matriz_confusion
dicc_tasas['REG_LOG']['W'] = rg_510.tasas_confusion


# Dataset Indians η = 1 y Népocas = 100
tasas_error = rg_1100.validacion(vc, dataset_indians, rg_1100, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE100']['N1']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE100']['N1']['I'] = rg_1100.matriz_confusion

# Dataset WDBC η = 1 y Népocas = 100
tasas_error = rg_1100.validacion(vc, dataset_wdbc, rg_1100, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE100']['N1']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE100']['N1']['W'] = rg_1100.matriz_confusion

# Dataset Indians η = 2 y Népocas = 100
tasas_error = rg_2100.validacion(vc, dataset_indians, rg_2100, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE100']['N2']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE100']['N2']['I'] = rg_2100.matriz_confusion

# Dataset WDBC η = 2 y Népocas = 100
tasas_error = rg_2100.validacion(vc, dataset_wdbc, rg_2100, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE100']['N2']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE100']['N2']['W'] = rg_2100.matriz_confusion

# Dataset Indians η = 0.5 y Népocas = 100
tasas_error = rg_5100.validacion(vc, dataset_indians, rg_5100, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE100']['N5']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_I'] = desviacion
dicc_matrices['REG_LOG']['NE100']['N5']['I'] = rg_5100.matriz_confusion

# Dataset WDBC η = 0.5 y Népocas = 100
tasas_error = rg_5100.validacion(vc, dataset_wdbc, rg_5100, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_med_desv['REG_LOG']['NE100']['N5']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_W'] = desviacion
dicc_matrices['REG_LOG']['NE100']['N5']['W'] = rg_5100.matriz_confusion

- Tras ejecutar todo lo anterior, se puede ver las siguientes tablas que almacenan las medias y desviaciones de las validaciones de cada modelo anterior para cada dataset existente.

In [ ]:
#print(dicc_med_desv)
# *Forma tabla*
# Dataset_x:       | E_μ | E_σ | E_μ | E_σ |...
#                 N|     |     |     |     |...
#                 N|     |     |     |     |...
#                 N|     |     |     |     |...
# E == Numero de épocas
# N == Valor de constante de aprendizaje
# μ == Media μ de las tasas de error 
# σ == Desviación típica σ de las tasas de error 
# x == Dataset Indians o WDBC

print("> Tabla con medias y desviaciones sobre dataset Indians\n")
print('    {6}   {0:3}   {6}   {1:3}   {6}   {2:3}  {6}   {3:3}  {6}  {4:3}  {6}  {5:3}  {6}'.format("E1_μ", "E1_σ", "E10_μ", "E10_σ", "E100_μ", "E100_σ", '|'))
print('0.5 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N5']['MED_I'] , dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_I'], dicc_med_desv['REG_LOG']['NE10']['N5']['MED_I'], dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_I'], dicc_med_desv['REG_LOG']['NE100']['N5']['MED_I'], dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_I'], '|'))
print('  1 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N1']['MED_I'] , dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_I'], dicc_med_desv['REG_LOG']['NE10']['N1']['MED_I'], dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_I'], dicc_med_desv['REG_LOG']['NE100']['N1']['MED_I'], dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_I'], '|'))
print('  2 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N2']['MED_I'] , dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_I'], dicc_med_desv['REG_LOG']['NE10']['N2']['MED_I'], dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_I'], dicc_med_desv['REG_LOG']['NE100']['N2']['MED_I'], dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_I'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset WDBC\n")
print('    {6}   {0:3}   {6}   {1:3}   {6}   {2:3}  {6}   {3:3}  {6}  {4:3}  {6}  {5:3}  {6}'.format("E1_μ", "E1_σ", "E10_μ", "E10_σ", "E100_μ", "E100_σ", '|'))
print('0.5 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N5']['MED_W'] , dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_W'], dicc_med_desv['REG_LOG']['NE10']['N5']['MED_W'], dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_W'], dicc_med_desv['REG_LOG']['NE100']['N5']['MED_W'], dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_W'], '|'))
print('  1 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N1']['MED_W'] , dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_W'], dicc_med_desv['REG_LOG']['NE10']['N1']['MED_W'], dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_W'], dicc_med_desv['REG_LOG']['NE100']['N1']['MED_W'], dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_W'], '|'))
print('  2 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N2']['MED_W'] , dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_W'], dicc_med_desv['REG_LOG']['NE10']['N2']['MED_W'], dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_W'], dicc_med_desv['REG_LOG']['NE100']['N2']['MED_W'], dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_W'], '|'))


- Observando los resultados nos hemos dado cuenta que en función del número de épocas, si este es mayor la precisión del modelo es peor. Ocurre lo mismo al revés, si el número de épocas es demasiado pequeño, también se obtiene una tasa de error mayor, en general. Por lo tanto, creemos que lo más óptimo es elegir un valor de número de épocas que no sea ni muy grande ni muy pequeño, en nuestro caso, en torno al 5-10. Esto es debido a que, cuantas más épocas se ejecuten en el entrenamiento, más grande serán los coeficientes de los pesos de w, derivando en una clasificación más estricta al ser la función sigmoidal muy abrupta, por lo que lleva a un mayor error. Y, si se ejecuta una sola época, por ejemplo, la frontera de decisión w apenas se ha podido situar correctamente, por lo que la clasificación con esos pesos de w es menos precisa. Lo óptimo es un valor equilibrado de número de épocas.


- En cuanto a la constante de aprendizaje, cuanto mayor es hemos notado que empeora la clasificación del modelo, y a un menor valor parece que la tasa de error es menor. Creemos que esto se debe a que si es mayor que 1, lo que se le resta al vector w en cada actualización es mayor y afecta más en sus pesos. Si es menor que 1, esta actualización es menor, más suave en los pesos. Por lo tanto, que las actualizaciones cambien de forma menos bruta conlleva que la frontera no varíe tanto y se adapte mejor en el espacio. Esto último lleva a que la clasificación sea mejor.


- Comparado con el clasificador de Vecinos Próximos obtenemos resultados parecidos para cada uno de los datasets. Para Indians, obtenemos una tasa de error del 30-35% con KNN y un 25-30% con Regresión Logística aproximadamente; y para wdbc, obtenemos un 0.05-0.08% con KNN y un 0.05% o menos con Regresión Logística aproximadamente. Por lo que notamos que Regresión Logística obtiene una clasificación un poco mejor.


- De nuevo, como en Vecinos Próximos, Regresión Logística también efectúa una muy buena clasificación con el dataset wdbc, aunque con un error un poco mejor al de KNN. Creemos que esto se debe a que el espacio de datos de wdbc está distribuido de manera muy clara, dejando en una zona la mayor parte de datos y en la otra las de la clase opuesta. Por lo tanto, la frontera lineal que se obtiene con este modelo divide muy bien ambos espacios de datos, por lo que la clasificación de nuevos datos es muy precisa. Entonces, este modelo también puede ser el modelo óptimo para este dataset, aunque lo pondríamos por delante de KNN si tuviéramos que elegir.


## APARTADO 3
En este tercer apartado hemos creado los modelos de Vecinos Próximos y de Regresión Logística mediante los métodos que proporciona la librería de Scikit-learn. Y, posteriormente, los hemos comparado con los nuestros anteriormente generados.

- Primero, nos centramos en Vecinos Próximos y utilizamos el método KNeighborsClassifier, que corresponde al modelo KNN en la librería de Scikit. A este método se le puede indicar el número K de vecinos mediante el valor del argumento n_neighbors; también, se le puede indicar la métrica para el cálculo de las distancias entre los datos clasificados y entrenados, que se introduce en el argumento metric; y, por último, si se modifica el atributo algorithm a 'brute' este modelo aplicará el cálculo de las distancias a todos los pares de datos, lo que nos interesa al ser igual que en nuestro modelo. Falta indicar que cuando se aplique la distancia Mahalanobis hay que introducir en metric_params la matriz inversa de covarianzas de los datos.


- Como hemos aplicado solamente la estrategia de particionado de Validación Cruzada, el entrenamiento y la clasificación la realizamos mediante el método cross_val_score de Scikit, indicándole 4 particiones.


- Pero antes, hemos aplicado un preprocesado a los datos para poder utilizarlos en las validaciones. Hemos normalizado los datos de cada dataset para poder comparar con los modelos con los datos sin normalizar. Este preprocesado lo hemos obtenido mediante StandardScaler() sobre el conjunto de datos sin clase, mediante un fit (cálculo de las medias y desviaciones de las  columnas) y posteriormente un transform (la normalización mediante los valores obtenidos en el fit). Entonces, los conjuntos de datos que utilizamos en este apartado son:

In [ ]:
# Datos sin normalizar
# X e y del dataset de Indians
X_ind = []
y_ind = []
for dato in dataset_indians.datos:
    y_ind.append(dato[-1])
    X_ind.append(dato[0:len(dato)-1])

# X e y del dataset de wdbc
X_w = []
y_w = []
for dato in dataset_wdbc.datos:
    y_w.append(dato[-1])
    X_w.append(dato[0:len(dato)-1])

    
# Datos normalizados
# Normalizamos los datos dataset Indians
scaler = StandardScaler()
scaler.fit(X_ind) # Calcula media y desviacion
X_train_ind_norm = scaler.transform(X_ind) # Normaliza

# Normalizamos los datos dataset WDBC
scaler = StandardScaler()
scaler.fit(X_w) # Calcula media y desviacion
X_train_w_norm = scaler.transform(X_w) # Normaliza

- A continuación, es necesario que creemos 4 matrices de covarianzas inversas para los modelos que utilicen Mahalanobis. Son cuatro porque diferenciamos por dataset y por preprocesado antes realizado. Estas son:

In [ ]:
# Calculo de matriz inversa dataset Indians y datos sin normalizar
datos_sin_clase = []
for dato in dataset_indians.datos:
    datos_sin_clase.append(dato[0:-1])
matriz_cov = np.cov(np.transpose(datos_sin_clase))
matriz_inversa_ind = np.linalg.inv(matriz_cov)

# Calculo de matriz inversa dataset WDBC y datos sin normalizar
datos_sin_clase = []
for dato in dataset_wdbc.datos:
    datos_sin_clase.append(dato[0:-1])
matriz_cov = np.cov(np.transpose(datos_sin_clase))
matriz_inversa_w = np.linalg.inv(matriz_cov)

# Calculo de matriz inversa dataset Indians y datos normalizados
matriz_cov = np.cov(np.transpose(X_train_ind_norm))
matriz_inversa_ind_norm = np.linalg.inv(matriz_cov)

# Calculo de matriz inversa dataset WDBC y datos normalizados
matriz_cov = np.cov(np.transpose(X_train_w_norm))
matriz_inversa_w_norm = np.linalg.inv(matriz_cov)

- Tras esto, ya creamos los distintos modelos de Vecinos Próximos que vamos a comparar con cada uno de los que hemos generado en el primer apartado. Vamos a utilizar el mismo diccionario de los apartados anteriores para almacenar las tasas de error medias y sus desviaciones. Por lo tanto:

In [ ]:
# Clasificador de KNN Distancia Euclidea con K = 1 para ambos datasets 
knn_scik_de_k1 = KNeighborsClassifier(n_neighbors=1, metric='euclidean', algorithm = 'brute')
# Clasificador de KNN Distancia Euclidea con K = 5 para ambos datasets 
knn_scik_de_k5 = KNeighborsClassifier(n_neighbors=5, metric='euclidean', algorithm = 'brute')
# Clasificador de KNN Distancia Euclidea con K = 11 para ambos datasets  
knn_scik_de_k11 = KNeighborsClassifier(n_neighbors=11, metric='euclidean', algorithm = 'brute')
# Clasificador de KNN Distancia Euclidea con K = 21 para ambos datasets 
knn_scik_de_k21 = KNeighborsClassifier(n_neighbors=21, metric='euclidean', algorithm = 'brute')

# Clasificador de KNN Distancia Manhattan con K = 1 para ambos datasets 
knn_scik_dmh_k1 = KNeighborsClassifier(n_neighbors=1, metric='manhattan', algorithm = 'brute')
# Clasificador de KNN Distancia Manhattan con K = 5 para ambos datasets  
knn_scik_dmh_k5 = KNeighborsClassifier(n_neighbors=5, metric='manhattan', algorithm = 'brute')
# Clasificador de KNN Distancia Manhattan con K = 11 para ambos datasets 
knn_scik_dmh_k11 = KNeighborsClassifier(n_neighbors=11, metric='manhattan', algorithm = 'brute')
# Clasificador de KNN Distancia Manhattan con K = 21 para ambos datasets  
knn_scik_dmh_k21 = KNeighborsClassifier(n_neighbors=21, metric='manhattan', algorithm = 'brute')

# Clasificador de KNN Distancia Mahalanobis con K = 1 y dataset Indians sin normalizar
knn_scik_dmb_k1_ind = KNeighborsClassifier(n_neighbors=1, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind})
# Clasificador de KNN Distancia Mahalanobis con K = 5 y dataset Indians sin normalizar 
knn_scik_dmb_k5_ind = KNeighborsClassifier(n_neighbors=5, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind})
# Clasificador de KNN Distancia Mahalanobis con K = 11 y dataset Indians sin normalizar 
knn_scik_dmb_k11_ind = KNeighborsClassifier(n_neighbors=11, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind})
# Clasificador de KNN Distancia Mahalanobis con K = 21 y dataset Indians sin normalizar
knn_scik_dmb_k21_ind = KNeighborsClassifier(n_neighbors=21, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind})

# Clasificador de KNN Distancia Mahalanobis con K = 1 y dataset WDBC sin normalizar
knn_scik_dmb_k1_w = KNeighborsClassifier(n_neighbors=1, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w})
# Clasificador de KNN Distancia Mahalanobis con K = 5 y dataset WDBC sin normalizar 
knn_scik_dmb_k5_w = KNeighborsClassifier(n_neighbors=5, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w})
# Clasificador de KNN Distancia Mahalanobis con K = 11 y dataset WDBC sin normalizar 
knn_scik_dmb_k11_w = KNeighborsClassifier(n_neighbors=11, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w})
# Clasificador de KNN Distancia Mahalanobis con K = 21 y dataset WDBC sin normalizar
knn_scik_dmb_k21_w = KNeighborsClassifier(n_neighbors=21, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w})

# Clasificador de KNN Distancia Mahalanobis con K = 1 y dataset Indians normalizado
knn_scik_dmb_k1_ind_norm = KNeighborsClassifier(n_neighbors=1, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind_norm})
# Clasificador de KNN Distancia Mahalanobis con K = 5 y dataset Indians normalizado
knn_scik_dmb_k5_ind_norm = KNeighborsClassifier(n_neighbors=5, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind_norm})
# Clasificador de KNN Distancia Mahalanobis con K = 11 y dataset Indians normalizado 
knn_scik_dmb_k11_ind_norm = KNeighborsClassifier(n_neighbors=11, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind_norm})
# Clasificador de KNN Distancia Mahalanobis con K = 21 y dataset Indians normalizado
knn_scik_dmb_k21_ind_norm = KNeighborsClassifier(n_neighbors=21, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_ind_norm})

# Clasificador de KNN Distancia Mahalanobis con K = 1 y dataset WDBC normalizado
knn_scik_dmb_k1_w_norm = KNeighborsClassifier(n_neighbors=1, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w_norm})
# Clasificador de KNN Distancia Mahalanobis con K = 5 y dataset WDBC normalizado 
knn_scik_dmb_k5_w_norm = KNeighborsClassifier(n_neighbors=5, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w_norm})
# Clasificador de KNN Distancia Mahalanobis con K = 11 y dataset WDBC normalizado
knn_scik_dmb_k11_w_norm = KNeighborsClassifier(n_neighbors=11, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w_norm})
# Clasificador de KNN Distancia Mahalanobis con K = 21 y dataset WDBC normalizado
knn_scik_dmb_k21_w_norm = KNeighborsClassifier(n_neighbors=21, metric='mahalanobis', algorithm = 'brute', metric_params={'VI': matriz_inversa_w_norm})


- Tras crear los modelos, procedemos a realizar las validaciones como en el apartado 1, para cada combinación de valores anterior de la siguiente forma:

In [ ]:
# Validaciones de los distintos modelos de Ks para cada dataset con Distancia Euclidea

# Dataset Indians K = 1 Datos Normalizados
scores = cross_val_score(knn_scik_de_k1, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 1 Datos Normalizados
scores = cross_val_score(knn_scik_de_k1, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_W'] = desviacion

# Dataset Indians K = 5 Datos Normalizados
scores = cross_val_score(knn_scik_de_k5, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 5 Datos Normalizados
scores = cross_val_score(knn_scik_de_k5, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_W'] = desviacion

# Dataset Indians K = 11 Datos Normalizados
scores = cross_val_score(knn_scik_de_k11, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 11 Datos Normalizados
scores = cross_val_score(knn_scik_de_k11, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_W'] = desviacion

# Dataset Indians K = 21 Datos Normalizados
scores = cross_val_score(knn_scik_de_k21, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['SN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 21 Datos Normalizados
scores = cross_val_score(knn_scik_de_k21, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['SN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_W'] = desviacion


# Dataset Indians K = 1 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k1, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 1 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k1, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_W'] = desviacion

# Dataset Indians K = 5 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k5, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 5 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k5, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_W'] = desviacion

# Dataset Indians K = 11 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k11, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 11 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k11, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_W'] = desviacion

# Dataset Indians K = 21 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k21, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['NN']['DE']['MED_I'] = media
dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_I'] = desviacion

# Dataset WDBC K = 21 Datos Sin Normalizar
scores = cross_val_score(knn_scik_de_k21, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['NN']['DE']['MED_W'] = media
dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_W'] = desviacion

In [ ]:
# Validaciones de los distintos modelos de Ks para cada dataset con Distancia Manhattan

# Dataset Indians K = 1 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k1, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 1 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k1, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_W'] = desviacion

# Dataset Indians K = 5 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k5, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 5 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k5, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_W'] = desviacion

# Dataset Indians K = 11 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k11, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 11 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k11, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_W'] = desviacion

# Dataset Indians K = 21 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k21, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 21 Datos Normalizados
scores = cross_val_score(knn_scik_dmh_k21, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_W'] = desviacion


# Dataset Indians K = 1 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k1, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 1 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k1, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_W'] = desviacion

# Dataset Indians K = 5 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k5, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 5 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k5, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_W'] = desviacion

# Dataset Indians K = 11 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k11, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 11 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k11, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_W'] = desviacion

# Dataset Indians K = 21 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k21, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_I'] = media
dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_I'] = desviacion

# Dataset WDBC K = 21 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmh_k21, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_W'] = media
dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_W'] = desviacion

In [ ]:
# Validaciones de los distintos modelos de Ks para cada dataset con Distancia Mahalanobis

# Dataset Indians K = 1 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k1_ind_norm, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 1 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k1_w_norm, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_W'] = desviacion

# Dataset Indians K = 5 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k5_ind_norm, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 5 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k5_w_norm, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_W'] = desviacion

# Dataset Indians K = 11 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k11_ind_norm, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 11 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k11_w_norm, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_W'] = desviacion

# Dataset Indians K = 21 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k21_ind_norm, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 21 Datos Normalizados
scores = cross_val_score(knn_scik_dmb_k21_w_norm, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_W'] = desviacion


# Dataset Indians K = 1 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k1_ind, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 1 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k1_w, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_W'] = desviacion

# Dataset Indians K = 5 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k5_ind, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 5 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k5_w, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_W'] = desviacion

# Dataset Indians K = 11 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k11_ind, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 11 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k11_w, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_W'] = desviacion

# Dataset Indians K = 21 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k21_ind, X_ind, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_I'] = media
dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_I'] = desviacion

# Dataset WDBC K = 21 Datos Sin Normalizar
scores = cross_val_score(knn_scik_dmb_k21_w, X_w, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_W'] = media
dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_W'] = desviacion

- Tras las ejecuciones anteriores, mostramos las mismas tablas que en el apartado 1 con los valores obtenidos de los modelos de Vecinos Próximos de Scikit-learn. Quedan así:

In [ ]:
#print(dicc_med_desv)
# *Forma tabla*
# Dataset_x_norm:        | K1_μ | K1_σ | K5_μ | K5_σ |...
#                      DE|      |      |      |      |...
#                     DMH|      |      |      |      |...
#                     DMB|      |      |      |      |...
# DE == Se usa estrategia Distancia Euclidea
# DMH == Se usa estrategia Distancia Manhattan
# DMB == Se usa estrategia Distancia Mahalanobis
# KN_μ == Media μ de las tasas de error sobre modelo con K = N
# KN_σ == Desviación típica σ de las tasas de error sobre modelo con K = N
# x == Dataset Indians o WDBC
# norm == Datos normalizados

print("> Tabla con medias y desviaciones sobre dataset Indians con datos normalizados de Scikit\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DE']['MED_I'] , dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_I'], dicc_med_desv['KNN']['K5']['SN']['DE']['MED_I'], dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_I'], dicc_med_desv['KNN']['K11']['SN']['DE']['MED_I'], dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_I'], dicc_med_desv['KNN']['K21']['SN']['DE']['MED_I'], dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_I'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_I'] , dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_I'], dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_I'], dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_I'], dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_I'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_I'] , dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_I'], dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_I'], dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_I'], dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_I'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset Indians sin datos normalizados de Scikit\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DE']['MED_I'] , dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_I'], dicc_med_desv['KNN']['K5']['NN']['DE']['MED_I'], dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_I'], dicc_med_desv['KNN']['K11']['NN']['DE']['MED_I'], dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_I'], dicc_med_desv['KNN']['K21']['NN']['DE']['MED_I'], dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_I'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_I'] , dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_I'], dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_I'], dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_I'], dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_I'], dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_I'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_I'] , dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_I'], dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_I'], dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_I'], dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_I'], dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_I'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset WDBC con datos normalizados de Scikit\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DE']['MED_W'] , dicc_med_desv['KNN']['K1']['SN']['DE']['DESV_W'], dicc_med_desv['KNN']['K5']['SN']['DE']['MED_W'], dicc_med_desv['KNN']['K5']['SN']['DE']['DESV_W'], dicc_med_desv['KNN']['K11']['SN']['DE']['MED_W'], dicc_med_desv['KNN']['K11']['SN']['DE']['DESV_W'], dicc_med_desv['KNN']['K21']['SN']['DE']['MED_W'], dicc_med_desv['KNN']['K21']['SN']['DE']['DESV_W'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMH']['MED_W'] , dicc_med_desv['KNN']['K1']['SN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K5']['SN']['DMH']['MED_W'], dicc_med_desv['KNN']['K5']['SN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K11']['SN']['DMH']['MED_W'], dicc_med_desv['KNN']['K11']['SN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K21']['SN']['DMH']['MED_W'], dicc_med_desv['KNN']['K21']['SN']['DMH']['DESV_W'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['SN']['DMB']['MED_W'] , dicc_med_desv['KNN']['K1']['SN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K5']['SN']['DMB']['MED_W'], dicc_med_desv['KNN']['K5']['SN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K11']['SN']['DMB']['MED_W'], dicc_med_desv['KNN']['K11']['SN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K21']['SN']['DMB']['MED_W'], dicc_med_desv['KNN']['K21']['SN']['DMB']['DESV_W'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset WDBC sin datos normalizados de Scikit\n")
print('    {8}   {0:3}   {8}   {1:3}   {8}   {2:3}   {8}   {3:3}   {8}   {4:3}  {8}   {5:3}  {8}   {6:3}  {8}   {7:3}  {8}'.format("K1_μ", "K1_σ", "K5_μ", "K5_σ", "K11_μ", "K11_σ", "K21_μ", "K21_σ", '|'))
print(' DE {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DE']['MED_W'] , dicc_med_desv['KNN']['K1']['NN']['DE']['DESV_W'], dicc_med_desv['KNN']['K5']['NN']['DE']['MED_W'], dicc_med_desv['KNN']['K5']['NN']['DE']['DESV_W'], dicc_med_desv['KNN']['K11']['NN']['DE']['MED_W'], dicc_med_desv['KNN']['K11']['NN']['DE']['DESV_W'], dicc_med_desv['KNN']['K21']['NN']['DE']['MED_W'], dicc_med_desv['KNN']['K21']['NN']['DE']['DESV_W'], '|'))
print('DMH {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMH']['MED_W'] , dicc_med_desv['KNN']['K1']['NN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K5']['NN']['DMH']['MED_W'], dicc_med_desv['KNN']['K5']['NN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K11']['NN']['DMH']['MED_W'], dicc_med_desv['KNN']['K11']['NN']['DMH']['DESV_W'], dicc_med_desv['KNN']['K21']['NN']['DMH']['MED_W'], dicc_med_desv['KNN']['K21']['NN']['DMH']['DESV_W'], '|'))
print('DMB {8} {0:2f} {8} {1:2f} {8} {2:2f} {8} {3:2f} {8} {4:2f} {8} {5:2f} {8} {6:2f} {8} {7:2f} {8}'.format(dicc_med_desv['KNN']['K1']['NN']['DMB']['MED_W'] , dicc_med_desv['KNN']['K1']['NN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K5']['NN']['DMB']['MED_W'], dicc_med_desv['KNN']['K5']['NN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K11']['NN']['DMB']['MED_W'], dicc_med_desv['KNN']['K11']['NN']['DMB']['DESV_W'], dicc_med_desv['KNN']['K21']['NN']['DMB']['MED_W'], dicc_med_desv['KNN']['K21']['NN']['DMB']['DESV_W'], '|'))


- Podemos ver que los resultados obtenidos mediante la librería Scikit-learn son prácticamente iguales a los que hemos obtenido en nuestra implementacién del clasificador de Vecinos Próximos. Por lo tanto, podemos asumir que nuestra implementación es correcta.


- Reforzando las ideas dichas en el apartado 1, se puede observar que se obtiene, en general, una tasa de error menor cuanto mayor es el valor de K. En cuanto, a las distancias, volvemos a tener unas tasas de error parecidas con Euclidea y Manhattan, que son menores a las de Mahalanobis (como ocurría en nuestra implementación). Así que creemos que realmente debe de estar bien nuestra implementación de Mahalanobis, al ser igual los resultados con Scikit (aunque seguimos pensando que en teoría debería ser mejor utilizar esta métrica).


- De nuevo, se obtienen tasas medias de error muy buenas con el dataset de wdbc, como nos ocurría en el apartado 1. Y con Indians también obtenemos las mismas, aunque no tan buenas como en el otro dataset. Y, además, el preprocesado de los datos en la normalización conduce a una mejor clasificación.

- A continuación, realizamos lo mismo pero para Regresión Logística. En este caso, Scikit-learn cuenta con dos implementaciones distintas: primero, tenemos el método LogisticRegression, que se encarga de generar un modelo de Regresión Logística que se encuentra regularizado por defecto, esto quiere decir que existe un regularizador, como en la versión MAP vista en teoría, que sirve para disminuir la complejidad del modelo (la nuestra es la versión por MV); y, segundo, tenemos el método SGDClassifier, que se encarga de generar modelos lineales estocásticos por descenso de gradiente, donde si adquiere como su función de pérdida (argumento loss) la logarítmica se convierte en Regresión Logística. Esta, como LogisticRegression, también se encuentra regularizada pero se diferencian, y es lo principal para nuestro caso, en que a SGDClassifier sí se le puede indicar cual es la constante de aprendizaje (argumento eta0 y argumento learning_rate a constante) a utilizar en el entrenamiento, por lo que vamos a utilizar este modelo para comparar con nuestro del apartado 2. A ambas se les puede indicar el número de épocas (argumento max_iter). 


- Falta indicar que ambos modelos se encuentran regularizados mediante un regularizador o penalización (en inglés penalty), y nuestro algoritmo implementado en el apartado 2 no lo posee, ya que este valor, como se ha dicho, disminuye la complejidad del modelo. Esta complejidad correspondería a los pesos de la frontera de decisión ω, que por cada iteración siempre van aumentando, por lo que a muy alto número de iteraciones sus pesos son de un orden grande. LogisticRegression sí nos permite no regularizar el modelo, pero no indicarle una constante de aprendizaje, por lo que aun así hemos elegido aplicar SGDClassifier como comparado con nuestra implementación.


- A continuación, creamos los modelos necesarios para la posterior validación y comparación del apartado 2. De esta manera:

In [ ]:
# Modelo de Regresión Logística con Népocas = 100
rg = LogisticRegression(max_iter= 100)

# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 1
rg_sgd_51 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 0.5, max_iter=1)
# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 1
rg_sgd_11 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 1, max_iter=1)
# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 1
rg_sgd_21 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 2, max_iter=1)

# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 10
rg_sgd_510 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 0.5, max_iter=10)
# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 10
rg_sgd_110 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 1, max_iter=10)
# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 10
rg_sgd_210 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 2, max_iter=10)

# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 100
rg_sgd_5100 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 0.5, max_iter=100)
# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 100
rg_sgd_1100 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 1, max_iter=100)
# Modelo SGDClassifier equivalente a Regresión Logística con η = 0.5 y Népocas = 100
rg_sgd_2100 = SGDClassifier(loss="log", learning_rate = "constant", eta0 = 2, max_iter=100)

- Tras crear los modelos, procedemos a realizar las validaciones como en el apartado 2, para cada combinación de valores anterior, con siempre datos normalizados, de la siguiente forma:

In [ ]:
# Dataset Indians η = 1 y Népocas = 1
scores = cross_val_score(rg_sgd_11, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE1']['N1']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_I'] = desviacion

# Dataset WDBC η = 1 y Népocas = 1
scores = cross_val_score(rg_sgd_11, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE1']['N1']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_W'] = desviacion

# Dataset Indians η = 2 y Népocas = 1
scores = cross_val_score(rg_sgd_21, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE1']['N2']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_I'] = desviacion

# Dataset WDBC η = 2 y Népocas = 1
scores = cross_val_score(rg_sgd_21, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE1']['N2']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_W'] = desviacion

# Dataset Indians η = 0.5 y Népocas = 1
scores = cross_val_score(rg_sgd_51, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE1']['N5']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_I'] = desviacion

# Dataset WDBC η = 0.5 y Népocas = 1
scores = cross_val_score(rg_sgd_51, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE1']['N5']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_W'] = desviacion


# Dataset Indians η = 1 y Népocas = 10
scores = cross_val_score(rg_sgd_110, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE10']['N1']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_I'] = desviacion

# Dataset WDBC η = 1 y Népocas = 10
scores = cross_val_score(rg_sgd_110, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE10']['N1']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_W'] = desviacion

# Dataset Indians η = 2 y Népocas = 10
scores = cross_val_score(rg_sgd_210, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE10']['N2']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_I'] = desviacion

# Dataset WDBC η = 2 y Népocas = 10
scores = cross_val_score(rg_sgd_210, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE10']['N2']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_W'] = desviacion

# Dataset Indians η = 0.5 y Népocas = 10
scores = cross_val_score(rg_sgd_510, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE10']['N5']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_I'] = desviacion

# Dataset WDBC η = 0.5 y Népocas = 10
scores = cross_val_score(rg_sgd_510, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE10']['N5']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_W'] = desviacion


# Dataset Indians η = 1 y Népocas = 100
scores = cross_val_score(rg_sgd_1100, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE100']['N1']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_I'] = desviacion

# Dataset WDBC η = 1 y Népocas = 100
scores = cross_val_score(rg_sgd_1100, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE100']['N1']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_W'] = desviacion

# Dataset Indians η = 2 y Népocas = 100
scores = cross_val_score(rg_sgd_2100, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE100']['N2']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_I'] = desviacion

# Dataset WDBC η = 2 y Népocas = 100
scores = cross_val_score(rg_sgd_2100, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE100']['N2']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_W'] = desviacion

# Dataset Indians η = 0.5 y Népocas = 100
scores = cross_val_score(rg_sgd_5100, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE100']['N5']['MED_I'] = media
dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_I'] = desviacion

# Dataset WDBC η = 0.5 y Népocas = 100
scores = cross_val_score(rg_sgd_5100, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
dicc_med_desv['REG_LOG']['NE100']['N5']['MED_W'] = media
dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_W'] = desviacion


- Puede que le aparezcan warnings, pero indican solo que el número de iteraciones es bajo para los casos 1 y 10, pero necesitamos ejecutarlos para compararlos con los del apartado 2. Y, aun así, no suponen ningún problema real porque los scores salen correctamente.


- Tras estas ejecuciones, mostramos las mismas tablas del apartado 2 pero con los datos de Scikit, de tal manera:

In [ ]:
#print(dicc_med_desv)
# *Forma tabla*
# Dataset_x:       | E_μ | E_σ | E_μ | E_σ |...
#                 N|     |     |     |     |...
#                 N|     |     |     |     |...
#                 N|     |     |     |     |...
# E == Numero de épocas
# N == Valor de constante de aprendizaje
# μ == Media μ de las tasas de error 
# σ == Desviación típica σ de las tasas de error 
# x == Dataset Indians o WDBC

print("> Tabla con medias y desviaciones sobre dataset Indians de SGDClassifier de Scikit\n")
print('    {6}   {0:3}   {6}   {1:3}   {6}   {2:3}  {6}   {3:3}  {6}  {4:3}  {6}  {5:3}  {6}'.format("E1_μ", "E1_σ", "E10_μ", "E10_σ", "E100_μ", "E100_σ", '|'))
print('0.5 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N5']['MED_I'] , dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_I'], dicc_med_desv['REG_LOG']['NE10']['N5']['MED_I'], dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_I'], dicc_med_desv['REG_LOG']['NE100']['N5']['MED_I'], dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_I'], '|'))
print('  1 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N1']['MED_I'] , dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_I'], dicc_med_desv['REG_LOG']['NE10']['N1']['MED_I'], dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_I'], dicc_med_desv['REG_LOG']['NE100']['N1']['MED_I'], dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_I'], '|'))
print('  2 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N2']['MED_I'] , dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_I'], dicc_med_desv['REG_LOG']['NE10']['N2']['MED_I'], dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_I'], dicc_med_desv['REG_LOG']['NE100']['N2']['MED_I'], dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_I'], '|'))

print("\n")

print("> Tabla con medias y desviaciones sobre dataset WDBC de SGDClassifier de Scikit\n")
print('    {6}   {0:3}   {6}   {1:3}   {6}   {2:3}  {6}   {3:3}  {6}  {4:3}  {6}  {5:3}  {6}'.format("E1_μ", "E1_σ", "E10_μ", "E10_σ", "E100_μ", "E100_σ", '|'))
print('0.5 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N5']['MED_W'] , dicc_med_desv['REG_LOG']['NE1']['N5']['DESV_W'], dicc_med_desv['REG_LOG']['NE10']['N5']['MED_W'], dicc_med_desv['REG_LOG']['NE10']['N5']['DESV_W'], dicc_med_desv['REG_LOG']['NE100']['N5']['MED_W'], dicc_med_desv['REG_LOG']['NE100']['N5']['DESV_W'], '|'))
print('  1 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N1']['MED_W'] , dicc_med_desv['REG_LOG']['NE1']['N1']['DESV_W'], dicc_med_desv['REG_LOG']['NE10']['N1']['MED_W'], dicc_med_desv['REG_LOG']['NE10']['N1']['DESV_W'], dicc_med_desv['REG_LOG']['NE100']['N1']['MED_W'], dicc_med_desv['REG_LOG']['NE100']['N1']['DESV_W'], '|'))
print('  2 {6} {0:2f} {6} {1:2f} {6} {2:2f} {6} {3:2f} {6} {4:2f} {6} {5:2f} {6}'.format(dicc_med_desv['REG_LOG']['NE1']['N2']['MED_W'] , dicc_med_desv['REG_LOG']['NE1']['N2']['DESV_W'], dicc_med_desv['REG_LOG']['NE10']['N2']['MED_W'], dicc_med_desv['REG_LOG']['NE10']['N2']['DESV_W'], dicc_med_desv['REG_LOG']['NE100']['N2']['MED_W'], dicc_med_desv['REG_LOG']['NE100']['N2']['DESV_W'], '|'))


- Comparando los resultados con los de nuestra implementación de Regresión Logística, podemos ver que estos son muy parecidos de nuevo. Por lo que asumimos que nuestra implementación de este clasificador ha sido la correcta.


- Reafirmando las ideas del apartado 2, se obtiene una mejor clasificación, no por mucho, cuando se elige un número de épocas ni muy grande ni muy pequeño (en nuestro caso el de 10). Y, en general, hemos obtenido una tase media de error menor con la constante de aprendizaje 0.5, frente a los valores de 1 y 2. 


- Se vuelve a obtener una muy buena clasificación con el dataset de wdbc.

- Y, para terminar este apartado, ejecutamos dos validaciones, una por cada dataset normalizado, para el modelo de LogisticRegression de Scikit-learn y así poder observar que los resultados obtenidos son igual de buenos que con SGDClassifier. Este modelo lo ejecutamos con sus valores por defecto, pero con 100 épocas. Por tanto:

In [ ]:
scores = cross_val_score(rg, X_train_ind_norm, y_ind, cv=4)
media,desviacion = calcula_med_desv(scores, True)
print("LogisticRegression con Indians media: " + str(media) + " desviacion tipica: " + str(desviacion))

scores = cross_val_score(rg, X_train_w_norm, y_w, cv=4)
media,desviacion = calcula_med_desv(scores, True)
print("LogisticRegression con WDBC media: " + str(media) + " desviacion tipica: " + str(desviacion))

- Vemos que con este otro modelo de la librería de Scikit-learn se obtienen unos resultados similares a los de SGDClassifier y  los de nuestra propia implementación. 

## APARTADO 4
En este último apartado se nos pidió extender el Análisis ROC que iniciamos en la primera práctica añadiendo los nuevos clasificadores implementados. En total haremos el Análisis ROC de los clasificadores Naive Bayes, Vecinos Próximos y Regresión Logística, con los datasets Indians y WDBC.

- Primero, mostramos las matrices de confusión calculadas en el apartado 1 para todas las combinaciones del clasificador de Vecinos Próximos con ambos datasets:

In [ ]:
#print(dicc_matrices)

#Datos normalizados

#Dataset Indians

#Distancia Euclidea para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Euclidea, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['SN']['DE']['I'][0], dicc_matrices['KNN']['K1']['SN']['DE']['I'][1], dicc_matrices['KNN']['K1']['SN']['DE']['I'][2], dicc_matrices['KNN']['K1']['SN']['DE']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Euclidea, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['SN']['DE']['I'][0], dicc_matrices['KNN']['K5']['SN']['DE']['I'][1], dicc_matrices['KNN']['K5']['SN']['DE']['I'][2], dicc_matrices['KNN']['K5']['SN']['DE']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Euclidea, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['SN']['DE']['I'][0], dicc_matrices['KNN']['K11']['SN']['DE']['I'][1], dicc_matrices['KNN']['K11']['SN']['DE']['I'][2], dicc_matrices['KNN']['K11']['SN']['DE']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Euclidea, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['SN']['DE']['I'][0], dicc_matrices['KNN']['K21']['SN']['DE']['I'][1], dicc_matrices['KNN']['K21']['SN']['DE']['I'][2], dicc_matrices['KNN']['K21']['SN']['DE']['I'][3],'|'))
print("\n")


#Distancia Manhattan para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Manhattan, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['SN']['DMH']['I'][0], dicc_matrices['KNN']['K1']['SN']['DMH']['I'][1], dicc_matrices['KNN']['K1']['SN']['DMH']['I'][2], dicc_matrices['KNN']['K1']['SN']['DMH']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Manhattan, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['SN']['DMH']['I'][0], dicc_matrices['KNN']['K5']['SN']['DMH']['I'][1], dicc_matrices['KNN']['K5']['SN']['DMH']['I'][2], dicc_matrices['KNN']['K5']['SN']['DMH']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Manhattan, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['SN']['DMH']['I'][0], dicc_matrices['KNN']['K11']['SN']['DMH']['I'][1], dicc_matrices['KNN']['K11']['SN']['DMH']['I'][2], dicc_matrices['KNN']['K11']['SN']['DMH']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Manhattan, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['SN']['DMH']['I'][0], dicc_matrices['KNN']['K21']['SN']['DMH']['I'][1], dicc_matrices['KNN']['K21']['SN']['DMH']['I'][2], dicc_matrices['KNN']['K21']['SN']['DMH']['I'][3],'|'))
print("\n")


#Distancia Mahalanobis para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Mahalanobis, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['SN']['DMB']['I'][0], dicc_matrices['KNN']['K1']['SN']['DMB']['I'][1], dicc_matrices['KNN']['K1']['SN']['DMB']['I'][2], dicc_matrices['KNN']['K1']['SN']['DMB']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Mahalanobis, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['SN']['DMB']['I'][0], dicc_matrices['KNN']['K5']['SN']['DMB']['I'][1], dicc_matrices['KNN']['K5']['SN']['DMB']['I'][2], dicc_matrices['KNN']['K5']['SN']['DMB']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Mahalanobis, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['SN']['DMB']['I'][0], dicc_matrices['KNN']['K11']['SN']['DMB']['I'][1], dicc_matrices['KNN']['K11']['SN']['DMB']['I'][2], dicc_matrices['KNN']['K11']['SN']['DMB']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Mahalanobis, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['SN']['DMB']['I'][0], dicc_matrices['KNN']['K21']['SN']['DMB']['I'][1], dicc_matrices['KNN']['K21']['SN']['DMB']['I'][2], dicc_matrices['KNN']['K21']['SN']['DMB']['I'][3],'|'))
print("\n")


#Dataset WDBC

#Distancia Euclidea para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Euclidea, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['SN']['DE']['W'][0], dicc_matrices['KNN']['K1']['SN']['DE']['W'][1], dicc_matrices['KNN']['K1']['SN']['DE']['W'][2], dicc_matrices['KNN']['K1']['SN']['DE']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Euclidea, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['SN']['DE']['W'][0], dicc_matrices['KNN']['K5']['SN']['DE']['W'][1], dicc_matrices['KNN']['K5']['SN']['DE']['W'][2], dicc_matrices['KNN']['K5']['SN']['DE']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Euclidea, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['SN']['DE']['W'][0], dicc_matrices['KNN']['K11']['SN']['DE']['W'][1], dicc_matrices['KNN']['K11']['SN']['DE']['W'][2], dicc_matrices['KNN']['K11']['SN']['DE']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Euclidea, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['SN']['DE']['W'][0], dicc_matrices['KNN']['K21']['SN']['DE']['W'][1], dicc_matrices['KNN']['K21']['SN']['DE']['W'][2], dicc_matrices['KNN']['K21']['SN']['DE']['W'][3],'|'))
print("\n")


#Distancia Manhattan para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Manhattan, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['SN']['DMH']['W'][0], dicc_matrices['KNN']['K1']['SN']['DMH']['W'][1], dicc_matrices['KNN']['K1']['SN']['DMH']['W'][2], dicc_matrices['KNN']['K1']['SN']['DMH']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Manhattan, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['SN']['DMH']['W'][0], dicc_matrices['KNN']['K5']['SN']['DMH']['W'][1], dicc_matrices['KNN']['K5']['SN']['DMH']['W'][2], dicc_matrices['KNN']['K5']['SN']['DMH']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Manhattan, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['SN']['DMH']['W'][0], dicc_matrices['KNN']['K11']['SN']['DMH']['W'][1], dicc_matrices['KNN']['K11']['SN']['DMH']['W'][2], dicc_matrices['KNN']['K11']['SN']['DMH']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Manhattan, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['SN']['DMH']['W'][0], dicc_matrices['KNN']['K21']['SN']['DMH']['W'][1], dicc_matrices['KNN']['K21']['SN']['DMH']['W'][2], dicc_matrices['KNN']['K21']['SN']['DMH']['W'][3],'|'))
print("\n")


#Distancia Mahalanobis para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Mahalanobis, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['SN']['DMB']['W'][0], dicc_matrices['KNN']['K1']['SN']['DMB']['W'][1], dicc_matrices['KNN']['K1']['SN']['DMB']['W'][2], dicc_matrices['KNN']['K1']['SN']['DMB']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Mahalanobis, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['SN']['DMB']['W'][0], dicc_matrices['KNN']['K5']['SN']['DMB']['W'][1], dicc_matrices['KNN']['K5']['SN']['DMB']['W'][2], dicc_matrices['KNN']['K5']['SN']['DMB']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Mahalanobis, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['SN']['DMB']['W'][0], dicc_matrices['KNN']['K11']['SN']['DMB']['W'][1], dicc_matrices['KNN']['K11']['SN']['DMB']['W'][2], dicc_matrices['KNN']['K11']['SN']['DMB']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Mahalanobis, y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['SN']['DMB']['W'][0], dicc_matrices['KNN']['K21']['SN']['DMB']['W'][1], dicc_matrices['KNN']['K21']['SN']['DMB']['W'][2], dicc_matrices['KNN']['K21']['SN']['DMB']['W'][3],'|'))
print("\n")

#Datos sin normalizar

#Dataset Indians

#Distancia Euclidea para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Euclidea, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['NN']['DE']['I'][0], dicc_matrices['KNN']['K1']['NN']['DE']['I'][1], dicc_matrices['KNN']['K1']['NN']['DE']['I'][2], dicc_matrices['KNN']['K1']['NN']['DE']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Euclidea, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['NN']['DE']['I'][0], dicc_matrices['KNN']['K5']['NN']['DE']['I'][1], dicc_matrices['KNN']['K5']['NN']['DE']['I'][2], dicc_matrices['KNN']['K5']['NN']['DE']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Euclidea, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['NN']['DE']['I'][0], dicc_matrices['KNN']['K11']['NN']['DE']['I'][1], dicc_matrices['KNN']['K11']['NN']['DE']['I'][2], dicc_matrices['KNN']['K11']['NN']['DE']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Euclidea, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['NN']['DE']['I'][0], dicc_matrices['KNN']['K21']['NN']['DE']['I'][1], dicc_matrices['KNN']['K21']['NN']['DE']['I'][2], dicc_matrices['KNN']['K21']['NN']['DE']['I'][3],'|'))
print("\n")


#Distancia Manhattan para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Manhattan, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['NN']['DMH']['I'][0], dicc_matrices['KNN']['K1']['NN']['DMH']['I'][1], dicc_matrices['KNN']['K1']['NN']['DMH']['I'][2], dicc_matrices['KNN']['K1']['NN']['DMH']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Manhattan, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['NN']['DMH']['I'][0], dicc_matrices['KNN']['K5']['NN']['DMH']['I'][1], dicc_matrices['KNN']['K5']['NN']['DMH']['I'][2], dicc_matrices['KNN']['K5']['NN']['DMH']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Manhattan, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['NN']['DMH']['I'][0], dicc_matrices['KNN']['K11']['NN']['DMH']['I'][1], dicc_matrices['KNN']['K11']['NN']['DMH']['I'][2], dicc_matrices['KNN']['K11']['NN']['DMH']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Manhattan, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['NN']['DMH']['I'][0], dicc_matrices['KNN']['K21']['NN']['DMH']['I'][1], dicc_matrices['KNN']['K21']['NN']['DMH']['I'][2], dicc_matrices['KNN']['K21']['NN']['DMH']['I'][3],'|'))
print("\n")


#Distancia Mahalanobis para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Mahalanobis, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['NN']['DMB']['I'][0], dicc_matrices['KNN']['K1']['NN']['DMB']['I'][1], dicc_matrices['KNN']['K1']['NN']['DMB']['I'][2], dicc_matrices['KNN']['K1']['NN']['DMB']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Mahalanobis, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['NN']['DMB']['I'][0], dicc_matrices['KNN']['K5']['NN']['DMB']['I'][1], dicc_matrices['KNN']['K5']['NN']['DMB']['I'][2], dicc_matrices['KNN']['K5']['NN']['DMB']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Mahalanobis, y datos sin normalizar sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['NN']['DMB']['I'][0], dicc_matrices['KNN']['K11']['NN']['DMB']['I'][1], dicc_matrices['KNN']['K11']['NN']['DMB']['I'][2], dicc_matrices['KNN']['K11']['NN']['DMB']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Mahalanobis, y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['NN']['DMB']['I'][0], dicc_matrices['KNN']['K21']['NN']['DMB']['I'][1], dicc_matrices['KNN']['K21']['NN']['DMB']['I'][2], dicc_matrices['KNN']['K21']['SN']['DMB']['I'][3],'|'))
print("\n")


#Dataset WDBC

#Distancia Euclidea para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Euclidea, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['NN']['DE']['W'][0], dicc_matrices['KNN']['K1']['NN']['DE']['W'][1], dicc_matrices['KNN']['K1']['NN']['DE']['W'][2], dicc_matrices['KNN']['K1']['NN']['DE']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Euclidea, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['NN']['DE']['W'][0], dicc_matrices['KNN']['K5']['NN']['DE']['W'][1], dicc_matrices['KNN']['K5']['NN']['DE']['W'][2], dicc_matrices['KNN']['K5']['NN']['DE']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Euclidea, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['NN']['DE']['W'][0], dicc_matrices['KNN']['K11']['NN']['DE']['W'][1], dicc_matrices['KNN']['K11']['NN']['DE']['W'][2], dicc_matrices['KNN']['K11']['NN']['DE']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Euclidea, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['NN']['DE']['W'][0], dicc_matrices['KNN']['K21']['NN']['DE']['W'][1], dicc_matrices['KNN']['K21']['NN']['DE']['W'][2], dicc_matrices['KNN']['K21']['NN']['DE']['W'][3],'|'))
print("\n")


#Distancia Manhattan para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Manhattan, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['NN']['DMH']['W'][0], dicc_matrices['KNN']['K1']['NN']['DMH']['W'][1], dicc_matrices['KNN']['K1']['NN']['DMH']['W'][2], dicc_matrices['KNN']['K1']['NN']['DMH']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Manhattan, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['NN']['DMH']['W'][0], dicc_matrices['KNN']['K5']['NN']['DMH']['W'][1], dicc_matrices['KNN']['K5']['NN']['DMH']['W'][2], dicc_matrices['KNN']['K5']['NN']['DMH']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Manhattan, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['NN']['DMH']['W'][0], dicc_matrices['KNN']['K11']['NN']['DMH']['W'][1], dicc_matrices['KNN']['K11']['NN']['DMH']['W'][2], dicc_matrices['KNN']['K11']['NN']['DMH']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Manhattan, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['NN']['DMH']['W'][0], dicc_matrices['KNN']['K21']['NN']['DMH']['W'][1], dicc_matrices['KNN']['K21']['NN']['DMH']['W'][2], dicc_matrices['KNN']['K21']['NN']['DMH']['W'][3],'|'))
print("\n")


#Distancia Mahalanobis para k =1, 5, 11 y 21
print("> Matriz de confusion para KNN con K = 1, distancia Mahalanobis, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K1']['NN']['DMB']['W'][0], dicc_matrices['KNN']['K1']['NN']['DMB']['W'][1], dicc_matrices['KNN']['K1']['NN']['DMB']['W'][2], dicc_matrices['KNN']['K1']['NN']['DMB']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 5, distancia Mahalanobis, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K5']['NN']['DMB']['W'][0], dicc_matrices['KNN']['K5']['NN']['DMB']['W'][1], dicc_matrices['KNN']['K5']['NN']['DMB']['W'][2], dicc_matrices['KNN']['K5']['NN']['DMB']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 11, distancia Mahalanobis, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K11']['NN']['DMB']['W'][0], dicc_matrices['KNN']['K11']['NN']['DMB']['W'][1], dicc_matrices['KNN']['K11']['NN']['DMB']['W'][2], dicc_matrices['KNN']['K11']['NN']['DMB']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para KNN con K = 21, distancia Mahalanobis, y datos sin normalizar sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['KNN']['K21']['NN']['DMB']['W'][0], dicc_matrices['KNN']['K21']['NN']['DMB']['W'][1], dicc_matrices['KNN']['K21']['NN']['DMB']['W'][2], dicc_matrices['KNN']['K21']['NN']['DMB']['W'][3],'|'))
print("\n")

- Segundo, mostramos las matrices de confusión calculadas en el apartado 2 para todas las combinaciones del clasificador de Regresión Logística con ambos datasets:

In [ ]:
#Datos normalizados

#Dataset Indians 

#Numero de epocas 1 con η = 0.5, 1, y 2
print("> Matriz de confusion para Regresión Logística con η = 0,5 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE1']['N5']['I'][0], dicc_matrices['REG_LOG']['NE1']['N5']['I'][1], dicc_matrices['REG_LOG']['NE1']['N5']['I'][2], dicc_matrices['REG_LOG']['NE1']['N5']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 1 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE1']['N1']['I'][0], dicc_matrices['REG_LOG']['NE1']['N1']['I'][1], dicc_matrices['REG_LOG']['NE1']['N1']['I'][2], dicc_matrices['REG_LOG']['NE1']['N1']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 2 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE1']['N2']['I'][0], dicc_matrices['REG_LOG']['NE1']['N2']['I'][1], dicc_matrices['REG_LOG']['NE1']['N2']['I'][2], dicc_matrices['REG_LOG']['NE1']['N2']['I'][3],'|'))
print("\n")

#Numero de epocas 10 con η = 0.5, 1, y 2
print("> Matriz de confusion para Regresión Logística con η = 0,5 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE10']['N5']['I'][0], dicc_matrices['REG_LOG']['NE10']['N5']['I'][1], dicc_matrices['REG_LOG']['NE10']['N5']['I'][2], dicc_matrices['REG_LOG']['NE10']['N5']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 1 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE10']['N1']['I'][0], dicc_matrices['REG_LOG']['NE10']['N1']['I'][1], dicc_matrices['REG_LOG']['NE10']['N1']['I'][2], dicc_matrices['REG_LOG']['NE10']['N1']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 2 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE10']['N2']['I'][0], dicc_matrices['REG_LOG']['NE10']['N2']['I'][1], dicc_matrices['REG_LOG']['NE10']['N2']['I'][2], dicc_matrices['REG_LOG']['NE10']['N2']['I'][3],'|'))
print("\n")

#Numero de epocas 100 con η = 0.5, 1, y 2
print("> Matriz de confusion para Regresión Logística con η = 0,5 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE100']['N5']['I'][0], dicc_matrices['REG_LOG']['NE100']['N5']['I'][1], dicc_matrices['REG_LOG']['NE100']['N5']['I'][2], dicc_matrices['REG_LOG']['NE100']['N5']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 1 y datos normailizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE100']['N1']['I'][0], dicc_matrices['REG_LOG']['NE100']['N1']['I'][1], dicc_matrices['REG_LOG']['NE100']['N1']['I'][2], dicc_matrices['REG_LOG']['NE100']['N1']['I'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 2 y datos normalizados sobre Indians:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE100']['N2']['I'][0], dicc_matrices['REG_LOG']['NE100']['N2']['I'][1], dicc_matrices['REG_LOG']['NE100']['N2']['I'][2], dicc_matrices['REG_LOG']['NE100']['N2']['I'][3],'|'))
print("\n")

#Dataset WDBC

#Numero de epocas 1 con η = 0.5, 1, y 2
print("> Matriz de confusion para Regresión Logística con η = 0,5 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE1']['N5']['W'][0], dicc_matrices['REG_LOG']['NE1']['N5']['W'][1], dicc_matrices['REG_LOG']['NE1']['N5']['W'][2], dicc_matrices['REG_LOG']['NE1']['N5']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 1 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE1']['N1']['W'][0], dicc_matrices['REG_LOG']['NE1']['N1']['W'][1], dicc_matrices['REG_LOG']['NE1']['N1']['W'][2], dicc_matrices['REG_LOG']['NE1']['N1']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 2 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE1']['N2']['W'][0], dicc_matrices['REG_LOG']['NE1']['N2']['W'][1], dicc_matrices['REG_LOG']['NE1']['N2']['W'][2], dicc_matrices['REG_LOG']['NE1']['N2']['W'][3],'|'))
print("\n")

#Numero de epocas 10 con η = 0.5, 1, y 2
print("> Matriz de confusion para Regresión Logística con η = 0,5 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE10']['N5']['W'][0], dicc_matrices['REG_LOG']['NE10']['N5']['W'][1], dicc_matrices['REG_LOG']['NE10']['N5']['W'][2], dicc_matrices['REG_LOG']['NE10']['N5']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 1 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE10']['N1']['W'][0], dicc_matrices['REG_LOG']['NE10']['N1']['W'][1], dicc_matrices['REG_LOG']['NE10']['N1']['W'][2], dicc_matrices['REG_LOG']['NE10']['N1']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 2 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE10']['N2']['W'][0], dicc_matrices['REG_LOG']['NE10']['N2']['W'][1], dicc_matrices['REG_LOG']['NE10']['N2']['W'][2], dicc_matrices['REG_LOG']['NE10']['N2']['W'][3],'|'))
print("\n")

#Numero de epocas 100 con η = 0.5, 1, y 2
print("> Matriz de confusion para Regresión Logística con η = 0,5 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE100']['N5']['W'][0], dicc_matrices['REG_LOG']['NE100']['N5']['W'][1], dicc_matrices['REG_LOG']['NE100']['N5']['W'][2], dicc_matrices['REG_LOG']['NE100']['N5']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 1 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE100']['N1']['W'][0], dicc_matrices['REG_LOG']['NE100']['N1']['W'][1], dicc_matrices['REG_LOG']['NE100']['N1']['W'][2], dicc_matrices['REG_LOG']['NE100']['N1']['W'][3],'|'))
print("\n")

print("> Matriz de confusion para Regresión Logística con η = 2 y datos normalizados sobre WDBC:\n")
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format("TP", "FP", "FN", "TN",'|'))
print(' {4}  {0:2}   {4}  {1:2}  {4}  {2:2}  {4}  {3:2}  {4}'.format(dicc_matrices['REG_LOG']['NE100']['N2']['W'][0], dicc_matrices['REG_LOG']['NE100']['N2']['W'][1], dicc_matrices['REG_LOG']['NE100']['N2']['W'][2], dicc_matrices['REG_LOG']['NE100']['N2']['W'][3],'|'))
print("\n")


- Tras observar todas las matrices de confusión correspondientes a cada una de las clasificaciones realizadas con nuestros modelos de Vecinos Próximos y Regresión Logística, podemos decir que hemos conseguido una buena generalización al tener muchos más verdaderos (Ts) que falsos (Fs) y con las predicciones de las clases repartidas coherentemente con el número real de datos con esas clases. Y en general, hemos notado que hay un poco más de clasificaciones negativas (Ns) que positivas (Ps), pero que debe de ser proporcional al número real de datos negativos y positivos de los datasets, por lo que es correto esto. 


- Como hemos visto en los apartados anteriores, en el dataset wdbc ambos clasificadores realizan una muy buena clasificación, que se ve reflejado en las matrices de confusión al haber muy pocas clasificaciones falsas frente a las clasificaciones correctas (verdaderas).


- Tercero, creamos el clasificador Naive Bayes realizado en la práctica anterior. Vamos a entrenar y clasificar el modelo con cada uno de los datasets:

In [ ]:
# Clasificador de NB con Laplace
nb_laplace = ClasificadorNaiveBayes(True)

# Dataset Indians
tasas_error = nb_laplace.validacion(vc, dataset_indians, nb_laplace, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_tasas['NB']['I'] = nb_laplace.tasas_confusion

print("NB con Indians media: " + str(media) + " desviacion tipica: " + str(desviacion))

#Dataset WDBC
tasas_error = nb_laplace.validacion(vc, dataset_wdbc, nb_laplace, None)
media,desviacion = calcula_med_desv(tasas_error, False)
dicc_tasas['NB']['W'] = nb_laplace.tasas_confusion

print("NB con WDBC media: " + str(media) + " desviacion tipica: " + str(desviacion))


- Podemos ver que nuestro clasificador de Naive Bayes (con Laplace) obtiene una tasa de error media del 23-25% para el dataset de Indians y una tasa del 0.05-0.07% para el dataset de wdbc. Parece que también podría ser un buen modelo final para el problema de clasificación del segundo dataset, como ha ocurrido con KNN y Regresión Logística.

- A continuación, vamos a dibujar en el espacio ROC los mejores clasificadores elegidos según las pruebas realizadas. Para el clasificador Naive Bayes hemos utilizado Laplace; para el de Vecinos Próximos K = 21, distancia Euclidea y datos normalizados; y para el de Regresión Logística 10 épocas y η = 0.5:

In [ ]:
#print(dicc_tasas)
# [TPR, FNR, FPR, TNR]

## Espacio ROC usando los clasificadores NB, KNN, RL sobre el dataset de Indians.
x = []
x.append(dicc_tasas['NB']['I'][2])
x.append(dicc_tasas['KNN']['I'][2])
x.append(dicc_tasas['REG_LOG']['I'][2])

y = []
y.append(dicc_tasas['NB']['I'][0])
y.append(dicc_tasas['KNN']['I'][0])
y.append(dicc_tasas['REG_LOG']['I'][0])

print("ROC sobre Indians: ")
print("Naive Bayes ==> FPR = " + str(x[0]) + " TPR = " + str(y[0]))
print("KNN ==> FPR = " + str(x[1]) + " TPR = " + str(y[1]))
print("Regresión Logística ==> FPR = " + str(x[2]) + " TPR = " + str(y[2]))

title('Espacio ROC Indians') 
xlabel('FPR')      
ylabel('TPR')    
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])

plot(x, y, 'o')                  
show()

## Espacio ROC usando los clasificadores NB, KNN, RL sobre el dataset de WDBC.
x = []
x.append(dicc_tasas['NB']['W'][2])
x.append(dicc_tasas['KNN']['W'][2])
x.append(dicc_tasas['REG_LOG']['W'][2])

y = []
y.append(dicc_tasas['NB']['W'][0])
y.append(dicc_tasas['KNN']['W'][0])
y.append(dicc_tasas['REG_LOG']['W'][0])

print("ROC sobre WDBC: ")
print("Naive Bayes ==> FPR = " + str(x[0]) + " TPR = " + str(y[0]))
print("KNN ==> FPR = " + str(x[1]) + " TPR = " + str(y[1]))
print("Regresión Logística ==> FPR = " + str(x[2]) + " TPR = " + str(y[2]))

title('Espacio ROC WDBC') 
xlabel('FPR')      
ylabel('TPR')    
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])

plot(x, y, 'o')                  
show()

- En los espacios ROC generados, el eje X es la tasa FPR y el eje Y es la tasa TPR. FPR es la tasa de falsos positivos sobre la suma de FP y TN, y TPR es la tasa de verdaderos positivos sobre la suma de TP y FN. Además, hemos dibujado la bisectriz del espacio ROC para comprobar que nuestros clasificadores funcionan correctamente, y para ello los puntos deben encontrarse por encima de la bisectriz. Si no fuera así, ese clasificador está haciendo más clasificaciones incorrectas que correctas.


- Para ambos datasets, podemos observar que los puntos de cada uno de los tres clasificadores se sitúan por encima de la bisectriz, por lo que, como hemos comentado, nuestras implementaciones de estos modelos realizan una clasificación correcta. Y, cabe añadir, que en el dataset de wdbc estos puntos se encuentran muy arriba a la izquierda de la gráfica, reafirmando que las predicciones han sido muy precisas para este dataset, y mejores que las obtenidas para Indians.


- Concretamente, el clasificador de Regresión Logística es el que se dibuja más cerca de la esquina superior izquierda (que corresponde al punto ideal de ninguna clasificación errónea). Luego, le sigue un poco más abajo el clasificador de Naive Bayes. Y, por último, el que se encuentra por debajo de ambos es el clasificador de Vecinos Próximos. Por lo tanto, podemos decir que el mejor clasificador para los dos datasets es Regresión Logística, y el menos bueno sería el de Vecinos Próximos.
